In [239]:
import ast
import sys
from typing import Union

import Polygon
import Polygon.IO
import clingo
import clingo.ast
import pyclipper
from clingo import PropagateInit
from clingo.ast import ProgramBuilder

In [240]:
PRINT_DETAIL_LEVEL = 0


def print_msg(value: object, detail_level=0, sep=' ', end='\n', file=sys.stdout, flush=True) -> None:
    """
    Prints message with given detail level to a stream. The higher the given detail_level, the fewer messages are printed
    :param values: Values to print
    :param detail_level: Given detail level
    :param sep: Seperator between values
    :param end: Prepended string
    :param file: The stream
    :param flush: If true flushes the output
    :return: None
    """
    if detail_level <= PRINT_DETAIL_LEVEL:
        print(value, sep=sep, end=end, file=file, flush=flush)

In [241]:
def _eval(t: clingo.TheoryTerm) -> Union[clingo.TheoryTerm, int, str]:
    """
    Unwraps number and function TheoryTerms. For numbers the number is returned as an int. For functions the string representation is returned.
    :param t: A clingo.TheoryTerm
    :return: Unwrapped value
    """
    if t.type is clingo.TheoryTermType.Symbol:
        return t
    if t.type is clingo.TheoryTermType.Number:
        return t.number
    if t.type is clingo.TheoryTermType.Function:
        return str(t)


In [242]:
def vertical_array(l, c):
    s = "%-" + str(c) + "s "
    n = 1
    i = 0
    while i < len(l) - 1:
        l.insert(i + 1, '|')
        i += (n + 1)
        s += "%s %-" + str(c) + "s "
    return s % tuple(l)

In [243]:
print(vertical_array(['one', 'two', 'three'], 1))

one | two | three 


# Checker

In [244]:
class Checker:

    def __init__(self):
        self._ctl = clingo.Control()
        self._map = []

    def backend(self):
        return self._ctl.backend()

    def add(self, guess_lit, check_lit):
        self._map.append((guess_lit, check_lit))

    def ground(self, check):
        with ProgramBuilder(self._ctl) as builder:
            for stm in check:
                builder.add(stm)
        self._ctl.ground([("base", [])])

    def pcheck(self, control, assume):
        assignment = control.assignment
        assumptions = []

        for guess_lit, check_lit in self._map:
            guess_truth = assignment.value(guess_lit)

            if check_lit in assume:
                assumptions.append(check_lit)

            elif guess_truth is not None:
                if guess_truth:
                    assumptions.append(check_lit)
                else:
                    assumptions.append(-check_lit)

        ret = self._ctl.solve(assumptions)
        if ret.satisfiable is not None:
            return ret.satisfiable

        raise RuntimeError("search interrupted")

In [245]:
theory_atom_names = ["union", "intersect", "diff", "buffer", "topology", "draw"]
spatial_function_names = ["union", "intersect", "diff", "buffer"]
spatial_relation_names = ["topology"]

In [246]:
spatial_artefact_names = ["constructed_surface", "walkable_surface", "transition_space", "site", "fall_space",
                          "movement_space"]

# Propagator

In [247]:

class GACPropagator(clingo.Propagator):

    def __init__(self, check):
        self._check = check
        self._checkers = []

        self._h2l = {}  # head theory atoms to solver literals
        self._b2l = {}  # head body atoms to solver literals
        self._h2b = {}
        self._b2h = {}

        self._p_lits = []  ## program literals
        self._s_lits = []  ## solver literals
        self._p2t = {}  ## program literals of theory atoms
        self._l2t = {}  ## solver literals of theory atoms
        self._s2p = {}  ## symbolic atoms to program literals

        self._geoms = {}
        self._rel = {}  ## spatial relations
        self._assigns = {}

    def init(self, init: PropagateInit) -> None:

        for _ in range(init.number_of_threads):
            checker = Checker()
            self._checkers.append(checker)

            with checker.backend() as backend:
                for atom in init.symbolic_atoms:

                    program_lit = atom.literal
                    solver_lit = init.solver_literal(program_lit)
                    init.add_watch(solver_lit)

                    if program_lit not in self._p_lits:
                        self._p_lits.append(program_lit)
                    if solver_lit not in self._s_lits:
                        self._s_lits.append(solver_lit)

                    guess_truth = init.assignment.value(solver_lit)

                    if guess_truth is False:
                        continue

                    check_lit = backend.add_atom(atom.symbol)

                    if guess_truth is True:
                        backend.add_rule([check_lit], [])
                    else:
                        backend.add_rule([check_lit], [], True)
                        checker.add(solver_lit, check_lit)

                    if atom.symbol.name in spatial_relation_names:
                        self._s2p.setdefault(atom.symbol, check_lit)

                for atom in init.theory_atoms:

                    program_lit = atom.literal
                    solver_lit = init.solver_literal(program_lit)
                    if program_lit not in self._p_lits:
                        self._p_lits.append(program_lit)
                    if solver_lit not in self._s_lits:
                        self._s_lits.append(solver_lit)

                    term = atom.term
                    op = term.name
                    loc = term.arguments[0].name

                    if op == "topology" and len(term.arguments) == 1:

                        x = str(atom.elements[0].terms[0])
                        y = str(atom.elements[0].terms[1])

                        self._p2t.setdefault(atom.literal, []).append((op, loc, x, y))
                        self._l2t.setdefault(solver_lit, []).append((op, loc, x, y))

                        if term.arguments[0].name == "head":
                            if (x, y) not in self._h2l.keys():
                                self._h2l.setdefault((x, y), []).append(solver_lit)

                        if term.arguments[0].name == "body":
                            b2l = self._b2l
                            if (x, y) not in b2l.keys():
                                b2l.setdefault((x, y), []).append(solver_lit)

                    if op in ("union", "intersect") and len(term.arguments) == 1:

                        ids = sorted([_eval(elt.terms[0]) for elt in atom.elements])
                        new_id = str(ids)
                        res = str(atom.guard[1])

                        self._p2t.setdefault(program_lit, []).append((op, loc, new_id, res))
                        self._l2t.setdefault(solver_lit, []).append((op, loc, new_id, res))

                        if loc == "head":
                            h2l = self._h2l
                            if (new_id, res) not in h2l.keys():
                                h2l.setdefault((new_id, res), []).append(solver_lit)

                        if loc == "body":
                            if (new_id, res) not in self._b2l.keys():
                                self._b2l.setdefault((new_id, res), []).append(solver_lit)

                    if op == "diff" and len(term.arguments) == 1:

                        x = str(atom.elements[0].terms[0].arguments[0])
                        y = str(atom.elements[0].terms[0].arguments[1])

                        new_id = str([x, y])
                        res = str(atom.guard[1])

                        self._p2t.setdefault(atom.literal, []).append((op, loc, new_id, res))
                        self._l2t.setdefault(solver_lit, []).append((op, loc, new_id, res))

                        if loc == "head":
                            if (new_id, res) not in self._h2l.keys():
                                self._h2l.setdefault((new_id, res), []).append(solver_lit)

                        if loc == "body":
                            if (new_id, res) not in self._b2l.keys():
                                self._b2l.setdefault((new_id, res), []).append(solver_lit)

                    if op == "buffer" and len(term.arguments) == 1:

                        id = str(atom.elements[0].terms[0])
                        cons = atom.elements[0].terms[1]

                        new_id = str([id, cons])
                        res = str(atom.guard[1])

                        self._p2t.setdefault(atom.literal, []).append((op, loc, new_id, res))
                        self._l2t.setdefault(solver_lit, []).append((op, loc, new_id, res))

                        if loc == "head":
                            if (new_id, res) not in self._h2l.keys():
                                self._h2l.setdefault((new_id, res), []).append(solver_lit)

                        if loc == "body":
                            if (new_id, res) not in self._b2l.keys():
                                self._b2l.setdefault((new_id, res), []).append(solver_lit)

                    if term.name == "draw" and len(term.arguments) == 1:

                        ids_ = [_eval(elt.terms[0]) for elt in atom.elements]
                        ids = sorted(ids_)

                        new_id = str(ids)

                        self._p2t.setdefault(atom.literal, []).append((op, loc, new_id))
                        self._l2t.setdefault(solver_lit, []).append((op, loc, new_id))

                        if loc == "head":
                            if new_id not in self._h2l.keys():
                                self._h2l.setdefault(new_id, []).append(solver_lit)

                        if loc == "body":
                            if new_id not in self._b2l.keys():
                                self._b2l.setdefault(new_id, []).append(solver_lit)

            h2l = self._h2l
            b2l = self._b2l
            heads = set(h2l)
            bodies = set(b2l)
            h2b = self._h2b
            b2h = self._b2h

            for t in heads.intersection(bodies):
                head = h2l[t][0]
                body = b2l[t][0]
                if head not in h2b:
                    h2b.setdefault(head, []).append(body)
                if body not in b2h:
                    b2h.setdefault(body, []).append(head)

            for body in b2h.keys():
                heads = b2h.get(body)
                for head in heads:
                    init.add_clause([-body, head])
                    init.add_clause([body, -head])

            checker.ground(self._check)

    def print_assignment(self, assignment):
        lits = self._s_lits
        literals = list(filter(lambda x: assignment.value(x) is not None, lits))
        truth_values = [lit if assignment.value(lit) else -lit for lit in literals]
        decision_levels = [assignment.level(lit) for lit in literals]
        print_msg("truth_values : " + str(truth_values), 4)
        print_msg("decision_levels : " + str(decision_levels), 4)
        return

    def inspect_assignment(self, assignment):

        print("decision level : ", assignment.decision_level)
        print("has_conflict : ", assignment.has_conflict)
        print("is_total : ", assignment.is_total)
        print("max_size : ", assignment.max_size)
        print("root_level : ", assignment.root_level)
        print("size : ", assignment.size)
        return

    def propagate(self, control, changes):

        print_msg("\n------------------------------------------------------Propagate---------------", 2)
        print_msg("active thread : " + str(control.thread_id), 2)
        print_msg("changes : " + str(changes), 3)

        assignment = control.assignment
        self.print_assignment(assignment)

        #self.check_partial(control)
        self.check_simple(control)

        print_msg("------------------------------------------------------Propagate---------------\n", 2)

    def undo(self, thread_id, assignment, changes):

        print_msg("\n::::::::::::::::::::::::::::::::::::::::::::::::::::::Undo::::::::::::::::::::", 2)
        print_msg("active thread : " + str(thread_id), 2)
        print_msg("undo : " + str(changes), 3)

        print_msg("::::::::::::::::::::Undo::::::::::::::::::::::::::::::::::::::::::::::::::::::\n", 2)

    def _head_atoms(self, literals, map):

        head_atoms = []
        body_atoms = []

        for l in literals:
            if l in map.keys():
                atoms = map.get(l)
                for atom in atoms:
                    if atom[1] == "head":
                        if atom[0] not in spatial_relation_names:
                            head_atoms.append(atom)
                        else:
                            body_atoms.append(atom)

        return head_atoms, body_atoms

    def _spatial_propagate(self, literals, map):

        eval, rel, assigns = self._geoms, self._rel, self._assigns
        print_msg("old eval : " + str(eval), 5)
        print_msg("old rel : " + str(rel), 5)

        reasoner = QSTR(eval, rel, assigns)
        head_atoms, body_atoms = self._head_atoms(literals, map)

        #print("propagate")
        #print(literals)
        #print(head_atoms)
        reasoner._evaluate(head_atoms)
        #reasoner._compute(body_atoms)
        assign = reasoner.get_assignment()
        relations = reasoner.get_rel()

        eval, rel, assigns = self._geoms, self._rel, self._assigns
        print_msg("new eval : " + str(eval), 5)
        print_msg("new rel : " + str(rel), 5)
        print_msg("new assign : " + str(assigns), 5)

        return assign, relations

    def get_assumptions(self, relations):

        assume = []
        s2p = self._s2p

        for p in relations.keys():
            r = relations.get(p)
            x, y = p[0], p[1]
            f = lambda s: clingo.Function(s, [])
            ext = clingo.Function("topology", [f(r), f(x), f(y)])
            print_msg("assumptions : " + str(ext), 3)
            if ext in s2p.keys():
                literal = s2p.get(ext)
                assume.append(literal)

        return assume

    def check_total(self, control):  ## check method as defined in clingo API

        #time.sleep(self.sleep)

        print_msg("\n;;;;;;;;;;;;;;;;Checking;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;", 2)

        print_msg("active thread : " + str(control.thread_id), 2)
        checker = self._checkers[control.thread_id]

        assignment = control.assignment
        self.print_assignment(assignment)

        if not checker.pcheck(control, []):
            conflict = []
            print_msg("(UNSATISFIABLE)", 2)
            print_msg('%-10s %s %-10s' % ("level", "|", "decision lit"), 4)

            for level in range(1, assignment.decision_level + 1):
                print_msg('%-10s %s %-10s' % (level, "|", assignment.decision(level)), 4)
                conflict.append(assignment.decision(level))
            print_msg("conflicts : " + str(conflict), 3)
            if conflict != []:
                control.add_nogood(conflict) and control.propagate()
                return

        literals = list(filter(lambda x: assignment.value(x) == True, self._s_lits))
        assign, relations = self._spatial_propagate(literals, self._l2t)
        print_msg("assign :" + " ".join(["{}={}".format(n, v) for n, v in assign.items()]), 5)
        assume = self.get_assumptions(relations)

        if not checker.pcheck(control, assume):
            conflict = []
            print_msg("(UNSATISFIABLE)", 2)
            print_msg('%-10s %s %-10s' % ("level", "|", "decision lit"), 4)

            for level in range(1, assignment.decision_level + 1):
                print_msg('%-10s %s %-10s' % (level, "|", assignment.decision(level)), 4)
                conflict.append(assignment.decision(level))
            print_msg("conflicts : " + str(conflict), 3)
            if conflict != []:
                control.add_nogood(conflict) and control.propagate()

            print_msg(";;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;Checking;;;;;;;;;;;;;;;;\n", 2)
            return

        else:

            print_msg("(SATISFIABLE)", 2)
            print_msg(";;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;Checking;;;;;;;;;;;;;;;;\n", 2)
            return

    def check_partial(self, control):

        #time.sleep(self.sleep)

        print_msg("\n;;;;;;;;;;;;;;;;Checking;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;", 2)

        print_msg("active thread : " + str(control.thread_id), 2)
        checker = self._checkers[control.thread_id]

        assignment = control.assignment
        self.print_assignment(assignment)

        if not checker.pcheck(control, []):
            conflict = []
            print_msg("(UNSATISFIABLE)", 2)
            print_msg('%-10s %s %-10s' % ("level", "|", "decision lit"), 4)

            for level in range(1, assignment.decision_level + 1):
                print_msg('%-10s %s %-10s' % (level, "|", assignment.decision(level)), 4)
                conflict.append(assignment.decision(level))
            print_msg("conflicts : " + str(conflict), 3)
            if conflict:
                control.add_nogood(conflict) and control.propagate()
                return

        literals = list(filter(lambda x: assignment.value(x) == True, self._s_lits))
        assign, relations = self._spatial_propagate(literals, self._l2t)
        print_msg("assign :" + " ".join(["{}={}".format(n, v) for n, v in assign.items()]), 5)
        assume = self.get_assumptions(relations)

        if not checker.pcheck(control, assume):
            conflict = []
            print_msg("(UNSATISFIABLE)", 2)
            print_msg('%-10s %s %-10s' % ("level", "|", "decision lit"), 4)

            for level in range(1, assignment.decision_level + 1):
                print_msg('%-10s %s %-10s' % (level, "|", assignment.decision(level)), 4)
                conflict.append(assignment.decision(level))
            print_msg("conflicts : " + str(conflict), 3)
            if conflict:
                control.add_nogood(conflict) and control.propagate()

            print_msg(";;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;Checking;;;;;;;;;;;;;;;;\n", 2)
            return

        else:

            print_msg("(SATISFIABLE)", 2)
            print_msg(";;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;Checking;;;;;;;;;;;;;;;;\n", 2)
            return

    def check_simple(self, control):

        #time.sleep(self.sleep)

        print_msg("\n;;;;;;;;;;;;;;;;Checking;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;", 2)

        print_msg("active thread : " + str(control.thread_id), 2)
        checker = self._checkers[control.thread_id]

        assignment = control.assignment
        #self.print_assignment(assignment)

        literals = list(filter(lambda x: assignment.value(x) == True, self._s_lits))
        assign, relations = self._spatial_propagate(literals, self._l2t)
        print_msg("assign :" + " ".join(["{}={}".format(n, v) for n, v in assign.items()]), 5)
        #assume = self.get_assumptions(relations)

        return

    def on_model(self, model):

        print_msg("\n================OnModel=======================================================", 2)
        print_msg("model thread : " + str(model.thread_id), 2)
        literals = list(filter(lambda x: model.is_true(x), self._p_lits))
        #head_atoms, body_atoms = self._head_atoms(literals, self._p2t)

        assignment, relations = self._spatial_propagate(literals, self._p2t)

        print("assignment :", " ".join(["{}={}".format(n, v) for n, v in assignment.items()]))

        #print("Answer: {}".format(model))
        #model.extend(clingo.Function("spatial", [var, value]) for var, value in assignment.items())
        print_msg("=======================================================OnModel================\n", 2)

# QRST

In [248]:
def _create_geom(c):
    c = ast.literal_eval(c)
    b, h = c[0], c[1]
    p = Polygon.Polygon(b[0])
    for i in range(len(b) - 1):
        p.addContour(b[i], 0)
    if len(h) == 0:
        return p
    else:
        p.addContour(h[0], 1)
        for i in range(len(h) - 1):
            p.addContour(h[i], 1)
        return p


def addPolygon(p):
    res = p[0]
    for i in range(1, len(p)):
        res = res + p[i]
    return res


def inflate(c, th):
    delta = th / 4
    coordinates = tuple(c)
    pco = pyclipper.PyclipperOffset(delta)
    coordinates_scaled = pyclipper.scale_to_clipper(coordinates, 1000)
    pco.AddPath(coordinates_scaled, pyclipper.JT_MITER, pyclipper.ET_CLOSEDPOLYGON)
    th_ = int(th * 1000)
    new_coordinates = pco.Execute(th_)
    new_coordinates_scaled = pyclipper.scale_from_clipper(new_coordinates, 1000)
    c_ = [tuple(new_coordinates_scaled[i]) for i in range(len(new_coordinates_scaled))]
    return c_


def deflate(c, th):
    delta = th / 4
    coordinates = tuple(c)
    pco = pyclipper.PyclipperOffset(delta)
    coordinates_scaled = pyclipper.scale_to_clipper(coordinates, 1000)
    pco.AddPath(coordinates_scaled, pyclipper.JT_MITER, pyclipper.ET_CLOSEDPOLYGON)
    th_ = int(th * 1000)
    new_coordinates = pco.Execute(-th_)
    new_coordinates_scaled = pyclipper.scale_from_clipper(new_coordinates, 1000)
    c_ = [tuple(new_coordinates_scaled[i]) for i in range(len(new_coordinates_scaled))]
    return c_


def buffer_(l, th):
    c = l[0]
    newc = inflate(c, th)
    p = Polygon.Polygon(newc[0])
    for i in range(1, len(newc)):
        p.addContour(newc[i], 0)
    for j in range(1, len(l)):
        h = l[j]
        newc = deflate(h, th)
        for k in range(len(newc)):
            p.addContour(newc[k], 1)
    return p


def _buffer(p, th):
    lp = []
    for i in range(len(p)):
        if not p.isHole(i):
            l = [p[i]]
            pc = Polygon.Polygon(p[i])
            temp = 0
            area = 0
            for j in range(len(p)):
                if p.isHole(j):
                    ph = Polygon.Polygon(p[j])
                    if pc.covers(ph):
                        if ph.area() > area:
                            temp = p[j]
                            area = ph.area()
            if temp != 0:
                l.append(temp)
            pi = buffer_(l, th)
            lp.append(pi)
    res = addPolygon(lp)
    return res

In [249]:
db = {
    '"3eHkYig055Ex7oFuhZVTBc"': '[[[(23.4011794330828, 24.673244124007), (21.5748227743323, 24.673244124007), (21.5748227743323, 22.7572879596117), (14.6511794330827, 22.7572879596117), (14.6511794330827, 34.0072879596117), (23.4011794330828, 34.0072879596117)]], []]',
    '"3eHkYig055Ex7oFuhZVTAV"': '[[[(32.6511794330828, 23.2572879596117), (25.7248227743324, 23.2572879596117), (25.7248227743324, 24.673244124007), (23.4011794330829, 24.673244124007), (23.4011794330829, 34.0072879596117), (32.6511794330828, 34.0072879596117)]], []]',
    '"3eHkYig055Ex7oFuhZVTAb"': '[[[(21.5748227743324, 21.823244124007), (23.9011794330828, 21.823244124007), (23.9011794330828, 12.0072879596117), (14.6511794330828, 12.0072879596117), (14.6511794330828, 22.7572879596117), (21.5748227743324, 22.7572879596117)]], []]',
    '"3eHkYig055Ex7oFuhZVT98"': '[[[(32.6511794330828, 12.0072879596116), (23.9011794330828, 12.0072879596116), (23.9011794330828, 21.823244124007), (25.7248227743323, 21.823244124007), (25.7248227743323, 23.2572879596116), (32.6511794330828, 23.2572879596116)]], []]',
    '"3eHkYig055Ex7oFuhZVIqN"': '[[[(25.4748227743324, 24.423244124007), (21.5748227743324, 24.423244124007), (21.5748227743324, 24.673244124007), (25.4748227743324, 24.673244124007)]], []]',
    '"3eHkYig055Ex7oFuhZVIqM"': '[[[(21.8248227743324, 21.823244124007), (21.5748227743324, 21.823244124007), (21.5748227743324, 24.423244124007), (21.8248227743324, 24.423244124007)]], []]',
    '"3eHkYig055Ex7oFuhZVIqf"': '[[[(25.7248227743324, 21.823244124007), (21.8248227743324, 21.823244124007), (21.8248227743324, 22.073244124007), (25.7248227743324, 22.073244124007)]], []]',
    '"3eHkYig055Ex7oFuhZVIqe"': '[[[(25.7248227743324, 22.073244124007), (25.4748227743324, 22.073244124007), (25.4748227743324, 24.673244124007), (25.7248227743324, 24.673244124007)]], []]',
    '"3eHkYig055Ex7oFuhZVInm"': '[[[(25.4748227743324, 24.423244124007), (21.5748227743324, 24.423244124007), (21.5748227743324, 24.673244124007), (25.4748227743324, 24.673244124007)]], []]',
    '"3eHkYig055Ex7oFuhZVInp"': '[[[(21.8248227743324, 21.823244124007), (21.5748227743324, 21.823244124007), (21.5748227743324, 24.423244124007), (21.8248227743324, 24.423244124007)]], []]',
    '"3eHkYig055Ex7oFuhZVIno"': '[[[(25.7248227743324, 21.823244124007), (21.8248227743324, 21.823244124007), (21.8248227743324, 22.073244124007), (25.7248227743324, 22.073244124007)]], []]',
    '"3eHkYig055Ex7oFuhZVInr"': '[[[(25.7248227743324, 22.073244124007), (25.4748227743324, 22.073244124007), (25.4748227743324, 24.673244124007), (25.7248227743324, 24.673244124007)]], []]',
    '"3eHkYig055Ex7oFuhZVI_Q"': '[[[(25.4748227743324, 24.423244124007), (21.5748227743324, 24.423244124007), (21.5748227743324, 24.673244124007), (25.4748227743324, 24.673244124007)]], []]',
    '"3eHkYig055Ex7oFuhZVI_T"': '[[[(21.8248227743324, 21.823244124007), (21.5748227743324, 21.823244124007), (21.5748227743324, 24.423244124007), (21.8248227743324, 24.423244124007)]], []]',
    '"3eHkYig055Ex7oFuhZVI_S"': '[[[(25.7248227743324, 21.823244124007), (21.8248227743324, 21.823244124007), (21.8248227743324, 22.073244124007), (25.7248227743324, 22.073244124007)]], []]',
    '"3eHkYig055Ex7oFuhZVI_V"': '[[[(25.7248227743324, 22.073244124007), (25.4748227743324, 22.073244124007), (25.4748227743324, 24.673244124007), (25.7248227743324, 24.673244124007)]], []]',
    '"3eHkYig055Ex7oFuhZVIya"': '[[[(25.4748227743324, 24.423244124007), (21.5748227743324, 24.423244124007), (21.5748227743324, 24.673244124007), (25.4748227743324, 24.673244124007)]], []]',
    '"3eHkYig055Ex7oFuhZVIyd"': '[[[(21.8248227743324, 21.823244124007), (21.5748227743324, 21.823244124007), (21.5748227743324, 24.423244124007), (21.8248227743324, 24.423244124007)]], []]',
    '"3eHkYig055Ex7oFuhZVIyc"': '[[[(25.7248227743324, 21.823244124007), (21.8248227743324, 21.823244124007), (21.8248227743324, 22.073244124007), (25.7248227743324, 22.073244124007)]], []]',
    '"3eHkYig055Ex7oFuhZVIyv"': '[[[(25.7248227743324, 22.073244124007), (25.4748227743324, 22.073244124007), (25.4748227743324, 24.673244124007), (25.7248227743324, 24.673244124007)]], []]',
    '"3eHkYig055Ex7oFuhZVIvF"': '[[[(25.4748227743324, 24.423244124007), (21.5748227743324, 24.423244124007), (21.5748227743324, 24.673244124007), (25.4748227743324, 24.673244124007)]], []]',
    '"3eHkYig055Ex7oFuhZVIvE"': '[[[(21.8248227743324, 21.823244124007), (21.5748227743324, 21.823244124007), (21.5748227743324, 24.423244124007), (21.8248227743324, 24.423244124007)]], []]',
    '"3eHkYig055Ex7oFuhZVIv1"': '[[[(25.7248227743324, 21.823244124007), (21.8248227743324, 21.823244124007), (21.8248227743324, 22.073244124007), (25.7248227743324, 22.073244124007)]], []]',
    '"3eHkYig055Ex7oFuhZVIv0"': '[[[(25.7248227743324, 22.073244124007), (25.4748227743324, 22.073244124007), (25.4748227743324, 24.673244124007), (25.7248227743324, 24.673244124007)]], []]',
    '"3eHkYig055Ex7oFuhZVId2"': '[[[(25.4748227743324, 24.423244124007), (21.5748227743324, 24.423244124007), (21.5748227743324, 24.673244124007), (25.4748227743324, 24.673244124007)]], []]',
    '"3eHkYig055Ex7oFuhZVId5"': '[[[(21.8248227743324, 21.823244124007), (21.5748227743324, 21.823244124007), (21.5748227743324, 24.423244124007), (21.8248227743324, 24.423244124007)]], []]',
    '"3eHkYig055Ex7oFuhZVId4"': '[[[(25.7248227743324, 21.823244124007), (21.8248227743324, 21.823244124007), (21.8248227743324, 22.073244124007), (25.7248227743324, 22.073244124007)]], []]',
    '"3eHkYig055Ex7oFuhZVId7"': '[[[(25.7248227743324, 22.073244124007), (25.4748227743324, 22.073244124007), (25.4748227743324, 24.673244124007), (25.7248227743324, 24.673244124007)]], []]',
    '"3eHkYig055Ex7oFuhZVIZL"': '[[[(21.5748227743324, 21.823244124007), (23.6511794330828, 21.823244124007), (23.6511794330828, 12.0072879596116), (14.6511794330828, 12.0072879596116), (14.6511794330828, 23.0072879596116), (21.5748227743324, 23.0072879596116)]], []]',
    '"3eHkYig055Ex7oFuhZVIYt"': '[[[(32.6511794330828, 12.0072879596116), (23.6511794330828, 12.0072879596116), (23.6511794330828, 21.823244124007), (25.7248227743324, 21.823244124007), (25.7248227743324, 23.0072879596116), (32.6511794330828, 23.0072879596116)]], []]',
    '"3eHkYig055Ex7oFuhZVIW2"': '[[[(32.6511794330828, 23.0072879596116), (25.7248227743324, 23.0072879596116), (25.7248227743324, 24.673244124007), (23.6511794330828, 24.673244124007), (23.6511794330828, 34.0072879596116), (32.6511794330828, 34.0072879596116)]], []]',
    '"2bxL$O5Cz3xOWd38LvRNOO"': '[[[(23.6511794330828, 24.673244124007), (21.5748227743324, 24.673244124007), (21.5748227743324, 23.0072879596116), (14.6511794330828, 23.0072879596116), (14.6511794330828, 34.0072879596116), (23.6511794330828, 34.0072879596116)]], []]',
    '"2AICztFX95Z8QtA12iaPQo"': '[[[(14.9511794330828, 33.5072879596117), (14.6511794330828, 33.5072879596117), (14.6511794330828, 34.0072879596117), (14.9511794330828, 34.0072879596117)]], []]',
    '"2AICztFX95Z8QtA12iaPQq"': '[[[(32.6511794330828, 33.5072879596117), (32.3511794330828, 33.5072879596117), (32.3511794330828, 34.0072879596117), (32.6511794330828, 34.0072879596117)]], []]',
    '"2AICztFX95Z8QtA12iaPQs"': '[[[(32.6511794330828, 12.0072879596116), (32.3511794330828, 12.0072879596116), (32.3511794330828, 12.5072879596116), (32.6511794330828, 12.5072879596116)]], []]',
    '"2AICztFX95Z8QtA12iaPR8"': '[[[(23.9011794330828, 12.0072879596117), (23.4011794330828, 12.0072879596117), (23.4011794330828, 12.3072879596117), (23.9011794330828, 12.3072879596117)]], []]',
    '"2AICztFX95Z8QtA12iaPRA"': '[[[(14.9511794330828, 12.0072879596117), (14.6511794330828, 12.0072879596117), (14.6511794330828, 12.5072879596117), (14.9511794330828, 12.5072879596117)]], []]',
    '"0N06MRuuX5WQjOvzkDy3l0"': '[[[(23.0248227743324, 20.549244124007), (22.2248227743324, 20.549244124007), (22.2248227743324, 23.597244124007), (23.0248227743324, 23.597244124007)]], []]',
    '"2AICztFX95Z8QtA12iaPRC"': '[[[(23.9011794330829, 33.7072879596117), (23.4011794330829, 33.7072879596117), (23.4011794330829, 34.0072879596117), (23.9011794330829, 34.0072879596117)]], []]',
    '"2AICztFX95Z8QtA12iaPRE"': '[[[(14.9511794330828, 28.2124985896816), (14.6511794330828, 28.2124985896816), (14.6511794330828, 28.7124985896816), (14.9511794330828, 28.7124985896816)]], []]',
    '"2AICztFX95Z8QtA12iaPR0"': '[[[(32.6511794330828, 28.2124985896815), (32.3511794330828, 28.2124985896815), (32.3511794330828, 28.7124985896815), (32.6511794330828, 28.7124985896815)]], []]',
    '"2AICztFX95Z8QtA12iaPR2"': '[[[(14.9511794330828, 17.2572879596116), (14.6511794330828, 17.2572879596116), (14.6511794330828, 17.7572879596116), (14.9511794330828, 17.7572879596116)]], []]',
    '"2AICztFX95Z8QtA12iaPR4"': '[[[(32.6511794330828, 17.2572879596116), (32.3511794330828, 17.2572879596116), (32.3511794330828, 17.7572879596116), (32.6511794330828, 17.7572879596116)]], []]',
    '"2AICztFX95Z8QtA12iaPR6"': '[[[(32.6511794330828, 22.7572879596116), (32.3511794330828, 22.7572879596116), (32.3511794330828, 23.2572879596116), (32.6511794330828, 23.2572879596116)]], []]',
    '"2AICztFX95Z8QtA12iaPRO"': '[[[(14.9511794330828, 22.7572879596117), (14.6511794330828, 22.7572879596117), (14.6511794330828, 23.2572879596117), (14.9511794330828, 23.2572879596117)]], []]',
    '"3eORFXfAH04e_QSwbI$PvJ"': '[[[(19.5261794330828, 12.0072879596117), (19.0261794330828, 12.0072879596117), (19.0261794330828, 12.3072879596117), (19.5261794330828, 12.3072879596117)]], []]',
    '"3eORFXfAH04e_QSwbI$P_Y"': '[[[(28.5261794330828, 12.0072879596116), (28.0261794330828, 12.0072879596116), (28.0261794330828, 12.3072879596116), (28.5261794330828, 12.3072879596116)]], []]',
    '"3eORFXfAH04e_QSwbI$P$5"': '[[[(19.5261794330828, 33.7072879596117), (19.0261794330828, 33.7072879596117), (19.0261794330828, 34.0072879596117), (19.5261794330828, 34.0072879596117)]], []]',
    '"3eORFXfAH04e_QSwbI$P$3"': '[[[(28.5261794330828, 33.7072879596117), (28.0261794330828, 33.7072879596117), (28.0261794330828, 34.0072879596117), (28.5261794330828, 34.0072879596117)]], []]',
    '"02NiuWFer1lA4a2StaofHI"': '[[[(14.9511794330828, 33.5072879596117), (14.6511794330828, 33.5072879596117), (14.6511794330828, 34.0072879596117), (14.9511794330828, 34.0072879596117)]], []]',
    '"02NiuWFer1lA4a2StaofHG"': '[[[(32.6511794330828, 33.5072879596117), (32.3511794330828, 33.5072879596117), (32.3511794330828, 34.0072879596117), (32.6511794330828, 34.0072879596117)]], []]',
    '"02NiuWFer1lA4a2StaofHM"': '[[[(32.6511794330828, 12.0072879596116), (32.3511794330828, 12.0072879596116), (32.3511794330828, 12.5072879596116), (32.6511794330828, 12.5072879596116)]], []]',
    '"02NiuWFer1lA4a2StaofHK"': '[[[(23.9011794330828, 12.0072879596117), (23.4011794330828, 12.0072879596117), (23.4011794330828, 12.3072879596117), (23.9011794330828, 12.3072879596117)]], []]',
    '"02NiuWFer1lA4a2StaofIg"': '[[[(14.9511794330828, 12.0072879596117), (14.6511794330828, 12.0072879596117), (14.6511794330828, 12.5072879596117), (14.9511794330828, 12.5072879596117)]], []]',
    '"02NiuWFer1lA4a2StaofIe"': '[[[(23.9011794330829, 33.7072879596117), (23.4011794330829, 33.7072879596117), (23.4011794330829, 34.0072879596117), (23.9011794330829, 34.0072879596117)]], []]',
    '"0N06MRuuX5WQjOvzkDy3lU"': '[[[(25.0748227743324, 20.549244124007), (24.2748227743324, 20.549244124007), (24.2748227743324, 23.597244124007), (25.0748227743324, 23.597244124007)]], []]',
    '"02NiuWFer1lA4a2StaofIk"': '[[[(14.9511794330828, 28.2124985896816), (14.6511794330828, 28.2124985896816), (14.6511794330828, 28.7124985896816), (14.9511794330828, 28.7124985896816)]], []]',
    '"02NiuWFer1lA4a2StaofIi"': '[[[(32.6511794330828, 28.2124985896815), (32.3511794330828, 28.2124985896815), (32.3511794330828, 28.7124985896815), (32.6511794330828, 28.7124985896815)]], []]',
    '"02NiuWFer1lA4a2StaofIY"': '[[[(14.9511794330828, 17.2572879596116), (14.6511794330828, 17.2572879596116), (14.6511794330828, 17.7572879596116), (14.9511794330828, 17.7572879596116)]], []]',
    '"02NiuWFer1lA4a2StaofIW"': '[[[(32.6511794330828, 17.2572879596116), (32.3511794330828, 17.2572879596116), (32.3511794330828, 17.7572879596116), (32.6511794330828, 17.7572879596116)]], []]',
    '"02NiuWFer1lA4a2StaofIc"': '[[[(32.6511794330828, 22.7572879596116), (32.3511794330828, 22.7572879596116), (32.3511794330828, 23.2572879596116), (32.6511794330828, 23.2572879596116)]], []]',
    '"02NiuWFer1lA4a2StaofIa"': '[[[(14.9511794330828, 22.7572879596117), (14.6511794330828, 22.7572879596117), (14.6511794330828, 23.2572879596117), (14.9511794330828, 23.2572879596117)]], []]',
    '"02NiuWFer1lA4a2StaofIw"': '[[[(19.5261794330828, 12.0072879596117), (19.0261794330828, 12.0072879596117), (19.0261794330828, 12.3072879596117), (19.5261794330828, 12.3072879596117)]], []]',
    '"02NiuWFer1lA4a2StaofIu"': '[[[(28.5261794330828, 12.0072879596116), (28.0261794330828, 12.0072879596116), (28.0261794330828, 12.3072879596116), (28.5261794330828, 12.3072879596116)]], []]',
    '"02NiuWFer1lA4a2StaofI_"': '[[[(19.5261794330828, 33.7072879596117), (19.0261794330828, 33.7072879596117), (19.0261794330828, 34.0072879596117), (19.5261794330828, 34.0072879596117)]], []]',
    '"02NiuWFer1lA4a2StaofIy"': '[[[(28.5261794330828, 33.7072879596117), (28.0261794330828, 33.7072879596117), (28.0261794330828, 34.0072879596117), (28.5261794330828, 34.0072879596117)]], []]',
    '"02NiuWFer1lA4a2StaofJx"': '[[[(14.9511794330828, 33.5072879596117), (14.6511794330828, 33.5072879596117), (14.6511794330828, 34.0072879596117), (14.9511794330828, 34.0072879596117)]], []]',
    '"02NiuWFer1lA4a2StaofJv"': '[[[(32.6511794330828, 33.5072879596117), (32.3511794330828, 33.5072879596117), (32.3511794330828, 34.0072879596117), (32.6511794330828, 34.0072879596117)]], []]',
    '"02NiuWFer1lA4a2StaofJ$"': '[[[(32.6511794330828, 12.0072879596116), (32.3511794330828, 12.0072879596116), (32.3511794330828, 12.5072879596116), (32.6511794330828, 12.5072879596116)]], []]',
    '"02NiuWFer1lA4a2StaofJz"': '[[[(23.9011794330828, 12.0072879596117), (23.4011794330828, 12.0072879596117), (23.4011794330828, 12.3072879596117), (23.9011794330828, 12.3072879596117)]], []]',
    '"02NiuWFer1lA4a2StaofJp"': '[[[(14.9511794330828, 12.0072879596117), (14.6511794330828, 12.0072879596117), (14.6511794330828, 12.5072879596117), (14.9511794330828, 12.5072879596117)]], []]',
    '"02NiuWFer1lA4a2StaofJn"': '[[[(23.9011794330829, 33.7072879596117), (23.4011794330829, 33.7072879596117), (23.4011794330829, 34.0072879596117), (23.9011794330829, 34.0072879596117)]], []]',
    '"02NiuWFer1lA4a2StaofJt"': '[[[(14.9511794330828, 28.2124985896816), (14.6511794330828, 28.2124985896816), (14.6511794330828, 28.7124985896816), (14.9511794330828, 28.7124985896816)]], []]',
    '"0N06MRuuX5WQjOvzkDy3l1"': '[[[(25.0748227743324, 22.899244124007), (24.2748227743324, 22.899244124007), (24.2748227743324, 25.947244124007), (25.0748227743324, 25.947244124007)]], []]',
    '"02NiuWFer1lA4a2StaofJr"': '[[[(32.6511794330828, 28.2124985896815), (32.3511794330828, 28.2124985896815), (32.3511794330828, 28.7124985896815), (32.6511794330828, 28.7124985896815)]], []]',
    '"02NiuWFer1lA4a2StaofJB"': '[[[(14.9511794330828, 17.2572879596116), (14.6511794330828, 17.2572879596116), (14.6511794330828, 17.7572879596116), (14.9511794330828, 17.7572879596116)]], []]',
    '"02NiuWFer1lA4a2StaofJ9"': '[[[(32.6511794330828, 17.2572879596116), (32.3511794330828, 17.2572879596116), (32.3511794330828, 17.7572879596116), (32.6511794330828, 17.7572879596116)]], []]',
    '"02NiuWFer1lA4a2StaofJF"': '[[[(32.6511794330828, 22.7572879596116), (32.3511794330828, 22.7572879596116), (32.3511794330828, 23.2572879596116), (32.6511794330828, 23.2572879596116)]], []]',
    '"02NiuWFer1lA4a2StaofJD"': '[[[(14.9511794330828, 22.7572879596117), (14.6511794330828, 22.7572879596117), (14.6511794330828, 23.2572879596117), (14.9511794330828, 23.2572879596117)]], []]',
    '"02NiuWFer1lA4a2StaofJ3"': '[[[(19.5261794330828, 12.0072879596117), (19.0261794330828, 12.0072879596117), (19.0261794330828, 12.3072879596117), (19.5261794330828, 12.3072879596117)]], []]',
    '"02NiuWFer1lA4a2StaofJ1"': '[[[(28.5261794330828, 12.0072879596116), (28.0261794330828, 12.0072879596116), (28.0261794330828, 12.3072879596116), (28.5261794330828, 12.3072879596116)]], []]',
    '"02NiuWFer1lA4a2StaofJ7"': '[[[(19.5261794330828, 33.7072879596117), (19.0261794330828, 33.7072879596117), (19.0261794330828, 34.0072879596117), (19.5261794330828, 34.0072879596117)]], []]',
    '"02NiuWFer1lA4a2StaofJ5"': '[[[(28.5261794330828, 33.7072879596117), (28.0261794330828, 33.7072879596117), (28.0261794330828, 34.0072879596117), (28.5261794330828, 34.0072879596117)]], []]',
    '"02NiuWFer1lA4a2StaofSC"': '[[[(14.9511794330828, 33.5072879596117), (14.6511794330828, 33.5072879596117), (14.6511794330828, 34.0072879596117), (14.9511794330828, 34.0072879596117)]], []]',
    '"02NiuWFer1lA4a2StaofS2"': '[[[(32.6511794330828, 33.5072879596117), (32.3511794330828, 33.5072879596117), (32.3511794330828, 34.0072879596117), (32.6511794330828, 34.0072879596117)]], []]',
    '"02NiuWFer1lA4a2StaofS0"': '[[[(32.6511794330828, 12.0072879596116), (32.3511794330828, 12.0072879596116), (32.3511794330828, 12.5072879596116), (32.6511794330828, 12.5072879596116)]], []]',
    '"02NiuWFer1lA4a2StaofS6"': '[[[(23.9011794330828, 12.0072879596117), (23.4011794330828, 12.0072879596117), (23.4011794330828, 12.3072879596117), (23.9011794330828, 12.3072879596117)]], []]',
    '"02NiuWFer1lA4a2StaofS4"': '[[[(14.9511794330828, 12.0072879596117), (14.6511794330828, 12.0072879596117), (14.6511794330828, 12.5072879596117), (14.9511794330828, 12.5072879596117)]], []]',
    '"02NiuWFer1lA4a2StaofSQ"': '[[[(23.9011794330829, 33.7072879596117), (23.4011794330829, 33.7072879596117), (23.4011794330829, 34.0072879596117), (23.9011794330829, 34.0072879596117)]], []]',
    '"02NiuWFer1lA4a2StaofSO"': '[[[(14.9511794330828, 28.2124985896816), (14.6511794330828, 28.2124985896816), (14.6511794330828, 28.7124985896816), (14.9511794330828, 28.7124985896816)]], []]',
    '"02NiuWFer1lA4a2StaofSU"': '[[[(32.6511794330828, 28.2124985896815), (32.3511794330828, 28.2124985896815), (32.3511794330828, 28.7124985896815), (32.6511794330828, 28.7124985896815)]], []]',
    '"0N06MRuuX5WQjOvzkDy0MP"': '[[[(23.0248227743324, 20.549244124007), (22.2248227743324, 20.549244124007), (22.2248227743324, 23.597244124007), (23.0248227743324, 23.597244124007)]], []]',
    '"02NiuWFer1lA4a2StaofSS"': '[[[(14.9511794330828, 17.2572879596116), (14.6511794330828, 17.2572879596116), (14.6511794330828, 17.7572879596116), (14.9511794330828, 17.7572879596116)]], []]',
    '"02NiuWFer1lA4a2StaofSI"': '[[[(32.6511794330828, 17.2572879596116), (32.3511794330828, 17.2572879596116), (32.3511794330828, 17.7572879596116), (32.6511794330828, 17.7572879596116)]], []]',
    '"02NiuWFer1lA4a2StaofSG"': '[[[(32.6511794330828, 22.7572879596116), (32.3511794330828, 22.7572879596116), (32.3511794330828, 23.2572879596116), (32.6511794330828, 23.2572879596116)]], []]',
    '"02NiuWFer1lA4a2StaofSM"': '[[[(14.9511794330828, 22.7572879596117), (14.6511794330828, 22.7572879596117), (14.6511794330828, 23.2572879596117), (14.9511794330828, 23.2572879596117)]], []]',
    '"02NiuWFer1lA4a2StaofSK"': '[[[(19.5261794330828, 12.0072879596117), (19.0261794330828, 12.0072879596117), (19.0261794330828, 12.3072879596117), (19.5261794330828, 12.3072879596117)]], []]',
    '"02NiuWFer1lA4a2StaofTg"': '[[[(28.5261794330828, 12.0072879596116), (28.0261794330828, 12.0072879596116), (28.0261794330828, 12.3072879596116), (28.5261794330828, 12.3072879596116)]], []]',
    '"02NiuWFer1lA4a2StaofTe"': '[[[(19.5261794330828, 33.7072879596117), (19.0261794330828, 33.7072879596117), (19.0261794330828, 34.0072879596117), (19.5261794330828, 34.0072879596117)]], []]',
    '"02NiuWFer1lA4a2StaofTk"': '[[[(28.5261794330828, 33.7072879596117), (28.0261794330828, 33.7072879596117), (28.0261794330828, 34.0072879596117), (28.5261794330828, 34.0072879596117)]], []]',
    '"02NiuWFer1lA4a2StaofTL"': '[[[(14.9511794330828, 33.5072879596117), (14.6511794330828, 33.5072879596117), (14.6511794330828, 34.0072879596117), (14.9511794330828, 34.0072879596117)]], []]',
    '"02NiuWFer1lA4a2StaofUh"': '[[[(32.6511794330828, 33.5072879596117), (32.3511794330828, 33.5072879596117), (32.3511794330828, 34.0072879596117), (32.6511794330828, 34.0072879596117)]], []]',
    '"02NiuWFer1lA4a2StaofUf"': '[[[(32.6511794330828, 12.0072879596116), (32.3511794330828, 12.0072879596116), (32.3511794330828, 12.5072879596116), (32.6511794330828, 12.5072879596116)]], []]',
    '"02NiuWFer1lA4a2StaofUl"': '[[[(23.9011794330828, 12.0072879596117), (23.4011794330828, 12.0072879596117), (23.4011794330828, 12.3072879596117), (23.9011794330828, 12.3072879596117)]], []]',
    '"02NiuWFer1lA4a2StaofUj"': '[[[(14.9511794330828, 12.0072879596117), (14.6511794330828, 12.0072879596117), (14.6511794330828, 12.5072879596117), (14.9511794330828, 12.5072879596117)]], []]',
    '"02NiuWFer1lA4a2StaofUZ"': '[[[(23.9011794330829, 33.7072879596117), (23.4011794330829, 33.7072879596117), (23.4011794330829, 34.0072879596117), (23.9011794330829, 34.0072879596117)]], []]',
    '"02NiuWFer1lA4a2StaofUX"': '[[[(14.9511794330828, 28.2124985896816), (14.6511794330828, 28.2124985896816), (14.6511794330828, 28.7124985896816), (14.9511794330828, 28.7124985896816)]], []]',
    '"02NiuWFer1lA4a2StaofUd"': '[[[(32.6511794330828, 28.2124985896815), (32.3511794330828, 28.2124985896815), (32.3511794330828, 28.7124985896815), (32.6511794330828, 28.7124985896815)]], []]',
    '"02NiuWFer1lA4a2StaofUb"': '[[[(14.9511794330828, 17.2572879596116), (14.6511794330828, 17.2572879596116), (14.6511794330828, 17.7572879596116), (14.9511794330828, 17.7572879596116)]], []]',
    '"0N06MRuuX5WQjOvzkDy0MN"': '[[[(25.0748227743324, 20.549244124007), (24.2748227743324, 20.549244124007), (24.2748227743324, 23.597244124007), (25.0748227743324, 23.597244124007)]], []]',
    '"02NiuWFer1lA4a2StaofUx"': '[[[(32.6511794330828, 17.2572879596116), (32.3511794330828, 17.2572879596116), (32.3511794330828, 17.7572879596116), (32.6511794330828, 17.7572879596116)]], []]',
    '"02NiuWFer1lA4a2StaofUv"': '[[[(32.6511794330828, 22.7572879596116), (32.3511794330828, 22.7572879596116), (32.3511794330828, 23.2572879596116), (32.6511794330828, 23.2572879596116)]], []]',
    '"02NiuWFer1lA4a2StaofU$"': '[[[(14.9511794330828, 22.7572879596117), (14.6511794330828, 22.7572879596117), (14.6511794330828, 23.2572879596117), (14.9511794330828, 23.2572879596117)]], []]',
    '"02NiuWFer1lA4a2StaofUz"': '[[[(19.5261794330828, 12.0072879596117), (19.0261794330828, 12.0072879596117), (19.0261794330828, 12.3072879596117), (19.5261794330828, 12.3072879596117)]], []]',
    '"02NiuWFer1lA4a2StaofUp"': '[[[(28.5261794330828, 12.0072879596116), (28.0261794330828, 12.0072879596116), (28.0261794330828, 12.3072879596116), (28.5261794330828, 12.3072879596116)]], []]',
    '"02NiuWFer1lA4a2StaofUn"': '[[[(19.5261794330828, 33.7072879596117), (19.0261794330828, 33.7072879596117), (19.0261794330828, 34.0072879596117), (19.5261794330828, 34.0072879596117)]], []]',
    '"02NiuWFer1lA4a2StaofUt"': '[[[(28.5261794330828, 33.7072879596117), (28.0261794330828, 33.7072879596117), (28.0261794330828, 34.0072879596117), (28.5261794330828, 34.0072879596117)]], []]',
    '"02NiuWFer1lA4a2StaofOP"': '[[[(14.9511794330828, 33.5072879596117), (14.6511794330828, 33.5072879596117), (14.6511794330828, 34.0072879596117), (14.9511794330828, 34.0072879596117)]], []]',
    '"02NiuWFer1lA4a2StaofOV"': '[[[(32.6511794330828, 33.5072879596117), (32.3511794330828, 33.5072879596117), (32.3511794330828, 34.0072879596117), (32.6511794330828, 34.0072879596117)]], []]',
    '"02NiuWFer1lA4a2StaofOT"': '[[[(32.6511794330828, 12.0072879596116), (32.3511794330828, 12.0072879596116), (32.3511794330828, 12.5072879596116), (32.6511794330828, 12.5072879596116)]], []]',
    '"02NiuWFer1lA4a2StaofOJ"': '[[[(23.9011794330828, 12.0072879596117), (23.4011794330828, 12.0072879596117), (23.4011794330828, 12.3072879596117), (23.9011794330828, 12.3072879596117)]], []]',
    '"02NiuWFer1lA4a2StaofOH"': '[[[(14.9511794330828, 12.0072879596117), (14.6511794330828, 12.0072879596117), (14.6511794330828, 12.5072879596117), (14.9511794330828, 12.5072879596117)]], []]',
    '"02NiuWFer1lA4a2StaofON"': '[[[(23.9011794330829, 33.7072879596117), (23.4011794330829, 33.7072879596117), (23.4011794330829, 34.0072879596117), (23.9011794330829, 34.0072879596117)]], []]',
    '"02NiuWFer1lA4a2StaofOL"': '[[[(14.9511794330828, 28.2124985896816), (14.6511794330828, 28.2124985896816), (14.6511794330828, 28.7124985896816), (14.9511794330828, 28.7124985896816)]], []]',
    '"02NiuWFer1lA4a2StaofPh"': '[[[(32.6511794330828, 28.2124985896815), (32.3511794330828, 28.2124985896815), (32.3511794330828, 28.7124985896815), (32.6511794330828, 28.7124985896815)]], []]',
    '"02NiuWFer1lA4a2StaofPf"': '[[[(14.9511794330828, 17.2572879596116), (14.6511794330828, 17.2572879596116), (14.6511794330828, 17.7572879596116), (14.9511794330828, 17.7572879596116)]], []]',
    '"02NiuWFer1lA4a2StaofPl"': '[[[(32.6511794330828, 17.2572879596116), (32.3511794330828, 17.2572879596116), (32.3511794330828, 17.7572879596116), (32.6511794330828, 17.7572879596116)]], []]',
    '"0N06MRuuX5WQjOvzkDy0MM"': '[[[(25.0748227743324, 22.899244124007), (24.2748227743324, 22.899244124007), (24.2748227743324, 25.947244124007), (25.0748227743324, 25.947244124007)]], []]',
    '"02NiuWFer1lA4a2StaofPj"': '[[[(32.6511794330828, 22.7572879596116), (32.3511794330828, 22.7572879596116), (32.3511794330828, 23.2572879596116), (32.6511794330828, 23.2572879596116)]], []]',
    '"02NiuWFer1lA4a2StaofPZ"': '[[[(14.9511794330828, 22.7572879596117), (14.6511794330828, 22.7572879596117), (14.6511794330828, 23.2572879596117), (14.9511794330828, 23.2572879596117)]], []]',
    '"02NiuWFer1lA4a2StaofPX"': '[[[(19.5261794330828, 12.0072879596117), (19.0261794330828, 12.0072879596117), (19.0261794330828, 12.3072879596117), (19.5261794330828, 12.3072879596117)]], []]',
    '"02NiuWFer1lA4a2StaofPd"': '[[[(28.5261794330828, 12.0072879596116), (28.0261794330828, 12.0072879596116), (28.0261794330828, 12.3072879596116), (28.5261794330828, 12.3072879596116)]], []]',
    '"02NiuWFer1lA4a2StaofPb"': '[[[(19.5261794330828, 33.7072879596117), (19.0261794330828, 33.7072879596117), (19.0261794330828, 34.0072879596117), (19.5261794330828, 34.0072879596117)]], []]',
    '"02NiuWFer1lA4a2StaofPx"': '[[[(28.5261794330828, 33.7072879596117), (28.0261794330828, 33.7072879596117), (28.0261794330828, 34.0072879596117), (28.5261794330828, 34.0072879596117)]], []]',
    '"02NiuWFer1lA4a2Staofbl"': '[[[(23.4011794330829, 24.673244124007), (21.5748227743324, 24.673244124007), (21.5748227743324, 22.7572879596117), (14.6511794330828, 22.7572879596117), (14.6511794330828, 34.0072879596117), (23.4011794330829, 34.0072879596117)]], []]',
    '"02NiuWFer1lA4a2Staofbc"': '[[[(32.6511794330828, 23.2572879596117), (25.7248227743324, 23.2572879596117), (25.7248227743324, 24.673244124007), (23.4011794330829, 24.673244124007), (23.4011794330829, 34.0072879596117), (32.6511794330828, 34.0072879596117)]], []]',
    '"02NiuWFer1lA4a2Staofbz"': '[[[(21.5748227743324, 21.823244124007), (23.9011794330828, 21.823244124007), (23.9011794330828, 12.0072879596117), (14.6511794330828, 12.0072879596117), (14.6511794330828, 22.7572879596117), (21.5748227743324, 22.7572879596117)]], []]',
    '"02NiuWFer1lA4a2Staofbq"': '[[[(32.6511794330828, 12.0072879596116), (23.9011794330828, 12.0072879596116), (23.9011794330828, 21.823244124007), (25.7248227743323, 21.823244124007), (25.7248227743323, 23.2572879596116), (32.6511794330828, 23.2572879596116)]], []]',
    '"02NiuWFer1lA4a2Staofc0"': '[[[(23.4011794330829, 24.673244124007), (21.5748227743324, 24.673244124007), (21.5748227743324, 22.7572879596117), (14.6511794330828, 22.7572879596117), (14.6511794330828, 34.0072879596117), (23.4011794330829, 34.0072879596117)]], []]',
    '"02NiuWFer1lA4a2StaofcV"': '[[[(32.6511794330828, 23.2572879596117), (25.7248227743324, 23.2572879596117), (25.7248227743324, 24.673244124007), (23.4011794330829, 24.673244124007), (23.4011794330829, 34.0072879596117), (32.6511794330828, 34.0072879596117)]], []]',
    '"02NiuWFer1lA4a2StaofcM"': '[[[(21.5748227743324, 21.823244124007), (23.9011794330828, 21.823244124007), (23.9011794330828, 12.0072879596117), (14.6511794330828, 12.0072879596117), (14.6511794330828, 22.7572879596117), (21.5748227743324, 22.7572879596117)]], []]',
    '"02NiuWFer1lA4a2Staofdj"': '[[[(32.6511794330828, 12.0072879596116), (23.9011794330828, 12.0072879596116), (23.9011794330828, 21.823244124007), (25.7248227743323, 21.823244124007), (25.7248227743323, 23.2572879596116), (32.6511794330828, 23.2572879596116)]], []]',
    '"02NiuWFer1lA4a2StaofWv"': '[[[(23.4011794330829, 24.673244124007), (21.5748227743324, 24.673244124007), (21.5748227743324, 22.7572879596117), (14.6511794330828, 22.7572879596117), (14.6511794330828, 34.0072879596117), (23.4011794330829, 34.0072879596117)]], []]',
    '"02NiuWFer1lA4a2StaofWm"': '[[[(32.6511794330828, 23.2572879596117), (25.7248227743324, 23.2572879596117), (25.7248227743324, 24.673244124007), (23.4011794330829, 24.673244124007), (23.4011794330829, 34.0072879596117), (32.6511794330828, 34.0072879596117)]], []]',
    '"02NiuWFer1lA4a2StaofWF"': '[[[(21.5748227743324, 21.823244124007), (23.9011794330828, 21.823244124007), (23.9011794330828, 12.0072879596117), (14.6511794330828, 12.0072879596117), (14.6511794330828, 22.7572879596117), (21.5748227743324, 22.7572879596117)]], []]',
    '"02NiuWFer1lA4a2StaofW6"': '[[[(32.6511794330829, 12.0072879596116), (23.9011794330829, 12.0072879596116), (23.9011794330829, 21.823244124007), (25.7248227743324, 21.823244124007), (25.7248227743324, 23.2572879596116), (32.6511794330829, 23.2572879596116)]], []]',
    '"02NiuWFer1lA4a2StaofXI"': '[[[(23.4011794330829, 24.673244124007), (21.5748227743324, 24.673244124007), (21.5748227743324, 22.7572879596117), (14.6511794330828, 22.7572879596117), (14.6511794330828, 34.0072879596117), (23.4011794330829, 34.0072879596117)]], []]',
    '"02NiuWFer1lA4a2StaofYf"': '[[[(32.6511794330828, 23.2572879596117), (25.7248227743324, 23.2572879596117), (25.7248227743324, 24.673244124007), (23.4011794330829, 24.673244124007), (23.4011794330829, 34.0072879596117), (32.6511794330828, 34.0072879596117)]], []]',
    '"02NiuWFer1lA4a2StaofYW"': '[[[(21.5748227743324, 21.823244124007), (23.9011794330828, 21.823244124007), (23.9011794330828, 12.0072879596117), (14.6511794330828, 12.0072879596117), (14.6511794330828, 22.7572879596117), (21.5748227743324, 22.7572879596117)]], []]',
    '"02NiuWFer1lA4a2StaofY$"': '[[[(32.6511794330829, 12.0072879596116), (23.9011794330829, 12.0072879596116), (23.9011794330829, 21.823244124007), (25.7248227743324, 21.823244124007), (25.7248227743324, 23.2572879596116), (32.6511794330829, 23.2572879596116)]], []]',
    '"02NiuWFer1lA4a2StaofZp"': '[[[(23.4011794330829, 24.673244124007), (21.5748227743324, 24.673244124007), (21.5748227743324, 22.7572879596117), (14.6511794330828, 22.7572879596117), (14.6511794330828, 34.0072879596117), (23.4011794330829, 34.0072879596117)]], []]',
    '"02NiuWFer1lA4a2StaofZA"': '[[[(32.6511794330828, 23.2572879596117), (25.7248227743324, 23.2572879596117), (25.7248227743324, 24.673244124007), (23.4011794330829, 24.673244124007), (23.4011794330829, 34.0072879596117), (32.6511794330828, 34.0072879596117)]], []]',
    '"02NiuWFer1lA4a2StaofZ1"': '[[[(21.5748227743324, 21.823244124007), (23.9011794330828, 21.823244124007), (23.9011794330828, 12.0072879596117), (14.6511794330828, 12.0072879596117), (14.6511794330828, 22.7572879596117), (21.5748227743324, 22.7572879596117)]], []]',
    '"02NiuWFer1lA4a2StaofZO"': '[[[(32.6511794330829, 12.0072879596116), (23.9011794330829, 12.0072879596116), (23.9011794330829, 21.823244124007), (25.7248227743324, 21.823244124007), (25.7248227743324, 23.2572879596116), (32.6511794330829, 23.2572879596116)]], []]',
    '"0RvmDCxjLDTADBjEF5c1$4"': '[[[(25.4748227743324, 24.423244124007), (21.5748227743324, 24.423244124007), (21.5748227743324, 24.673244124007), (25.4748227743324, 24.673244124007)]], []]',
    '"0RvmDCxjLDTADBjEF5c1$5"': '[[[(21.8248227743324, 21.823244124007), (21.5748227743324, 21.823244124007), (21.5748227743324, 24.423244124007), (21.8248227743324, 24.423244124007)]], []]',
    '"0RvmDCxjLDTADBjEF5c1$A"': '[[[(25.7248227743324, 21.823244124007), (21.8248227743324, 21.823244124007), (21.8248227743324, 22.073244124007), (25.7248227743324, 22.073244124007)]], []]',
    '"0RvmDCxjLDTADBjEF5c1$B"': '[[[(25.7248227743324, 22.073244124007), (25.4748227743324, 22.073244124007), (25.4748227743324, 24.673244124007), (25.7248227743324, 24.673244124007)]], []]',
    '"0RvmDCxjLDTADBjEF5c1_C"': '[[[(25.7248227743324, 21.823244124007), (21.5748227743324, 21.823244124007), (21.5748227743324, 24.673244124007), (25.7248227743324, 24.673244124007)]], []]',
    '"0N06MRuuX5WQjOvyYDy0T6"': '[[[(25.4748227743324, 24.423244124007), (21.5748227743324, 24.423244124007), (21.5748227743324, 24.673244124007), (25.4748227743324, 24.673244124007)]], []]',
    '"0N06MRuuX5WQjOvyYDy0T9"': '[[[(21.8248227743324, 21.823244124007), (21.5748227743324, 21.823244124007), (21.5748227743324, 24.423244124007), (21.8248227743324, 24.423244124007)]], []]',
    '"0N06MRuuX5WQjOvyYDy0T8"': '[[[(25.7248227743324, 21.823244124007), (21.8248227743324, 21.823244124007), (21.8248227743324, 22.073244124007), (25.7248227743324, 22.073244124007)]], []]',
    '"0N06MRuuX5WQjOvyYDy0TB"': '[[[(25.7248227743324, 22.073244124007), (25.4748227743324, 22.073244124007), (25.4748227743324, 24.673244124007), (25.7248227743324, 24.673244124007)]], []]',
    '"0N06MRuuX5WQjOvyYDy0TF"': '[[[(14.9511794330828, 33.5072879596117), (14.6511794330828, 33.5072879596117), (14.6511794330828, 34.0072879596117), (14.9511794330828, 34.0072879596117)]], []]',
    '"0N06MRuuX5WQjOvyYDy0Qn"': '[[[(32.6511794330828, 33.5072879596117), (32.3511794330828, 33.5072879596117), (32.3511794330828, 34.0072879596117), (32.6511794330828, 34.0072879596117)]], []]',
    '"0N06MRuuX5WQjOvyYDy0Qp"': '[[[(32.6511794330828, 12.0072879596116), (32.3511794330828, 12.0072879596116), (32.3511794330828, 12.5072879596116), (32.6511794330828, 12.5072879596116)]], []]',
    '"0N06MRuuX5WQjOvyYDy0Qr"': '[[[(23.9011794330828, 12.0072879596117), (23.4011794330828, 12.0072879596117), (23.4011794330828, 12.3072879596117), (23.9011794330828, 12.3072879596117)]], []]',
    '"0N06MRuuX5WQjOvyYDy0Qt"': '[[[(14.9511794330828, 12.0072879596117), (14.6511794330828, 12.0072879596117), (14.6511794330828, 12.5072879596117), (14.9511794330828, 12.5072879596117)]], []]',
    '"0N06MRuuX5WQjOvyYDy0Qv"': '[[[(23.9011794330829, 33.7072879596117), (23.4011794330829, 33.7072879596117), (23.4011794330829, 34.0072879596117), (23.9011794330829, 34.0072879596117)]], []]',
    '"0N06MRuuX5WQjOvyYDy0Qx"': '[[[(14.9511794330828, 28.2124985896816), (14.6511794330828, 28.2124985896816), (14.6511794330828, 28.7124985896816), (14.9511794330828, 28.7124985896816)]], []]',
    '"0N06MRuuX5WQjOvyYDy0Qz"': '[[[(32.6511794330828, 28.2124985896815), (32.3511794330828, 28.2124985896815), (32.3511794330828, 28.7124985896815), (32.6511794330828, 28.7124985896815)]], []]',
    '"0N06MRuuX5WQjOvyYDy0Q$"': '[[[(14.9511794330828, 17.2572879596116), (14.6511794330828, 17.2572879596116), (14.6511794330828, 17.7572879596116), (14.9511794330828, 17.7572879596116)]], []]',
    '"0N06MRuuX5WQjOvyYDy0QX"': '[[[(32.6511794330828, 17.2572879596116), (32.3511794330828, 17.2572879596116), (32.3511794330828, 17.7572879596116), (32.6511794330828, 17.7572879596116)]], []]',
    '"0N06MRuuX5WQjOvyYDy0QZ"': '[[[(32.6511794330828, 22.7572879596116), (32.3511794330828, 22.7572879596116), (32.3511794330828, 23.2572879596116), (32.6511794330828, 23.2572879596116)]], []]',
    '"0N06MRuuX5WQjOvzkDy0OZ"': '[[[(23.0248227743324, 20.549244124007), (22.2248227743324, 20.549244124007), (22.2248227743324, 23.597244124007), (23.0248227743324, 23.597244124007)]], []]',
    '"0N06MRuuX5WQjOvyYDy0Qb"': '[[[(14.9511794330828, 22.7572879596117), (14.6511794330828, 22.7572879596117), (14.6511794330828, 23.2572879596117), (14.9511794330828, 23.2572879596117)]], []]',
    '"0N06MRuuX5WQjOvyYDy0Qd"': '[[[(19.5261794330828, 12.0072879596117), (19.0261794330828, 12.0072879596117), (19.0261794330828, 12.3072879596117), (19.5261794330828, 12.3072879596117)]], []]',
    '"0N06MRuuX5WQjOvyYDy0Qf"': '[[[(28.5261794330828, 12.0072879596116), (28.0261794330828, 12.0072879596116), (28.0261794330828, 12.3072879596116), (28.5261794330828, 12.3072879596116)]], []]',
    '"0N06MRuuX5WQjOvyYDy0Qh"': '[[[(19.5261794330828, 33.7072879596117), (19.0261794330828, 33.7072879596117), (19.0261794330828, 34.0072879596117), (19.5261794330828, 34.0072879596117)]], []]',
    '"0N06MRuuX5WQjOvyYDy0Qj"': '[[[(28.5261794330828, 33.7072879596117), (28.0261794330828, 33.7072879596117), (28.0261794330828, 34.0072879596117), (28.5261794330828, 34.0072879596117)]], []]',
    '"0N06MRuuX5WQjOvyYDy0QH"': '[[[(23.4011794330829, 24.673244124007), (21.5748227743324, 24.673244124007), (21.5748227743324, 22.7572879596117), (14.6511794330828, 22.7572879596117), (14.6511794330828, 34.0072879596117), (23.4011794330829, 34.0072879596117)]], []]',
    '"0N06MRuuX5WQjOvyYDy0QO"': '[[[(32.6511794330828, 23.2572879596117), (25.7248227743324, 23.2572879596117), (25.7248227743324, 24.673244124007), (23.4011794330829, 24.673244124007), (23.4011794330829, 34.0072879596117), (32.6511794330828, 34.0072879596117)]], []]',
    '"0N06MRuuX5WQjOvyYDy0Q3"': '[[[(21.5748227743324, 21.823244124007), (23.9011794330828, 21.823244124007), (23.9011794330828, 12.0072879596117), (14.6511794330828, 12.0072879596117), (14.6511794330828, 22.7572879596117), (21.5748227743324, 22.7572879596117)]], []]',
    '"0N06MRuuX5WQjOvyYDy0QA"': '[[[(32.6511794330829, 12.0072879596116), (23.9011794330829, 12.0072879596116), (23.9011794330829, 21.823244124007), (25.7248227743324, 21.823244124007), (25.7248227743324, 23.2572879596116), (32.6511794330829, 23.2572879596116)]], []]',
    '"0N06MRuuX5WQjOvyYDy0Oz"': '[[[(25.4748227743324, 24.423244124007), (21.5748227743324, 24.423244124007), (21.5748227743324, 24.673244124007), (25.4748227743324, 24.673244124007)]], []]',
    '"0N06MRuuX5WQjOvyYDy0Oy"': '[[[(21.8248227743324, 21.823244124007), (21.5748227743324, 21.823244124007), (21.5748227743324, 24.423244124007), (21.8248227743324, 24.423244124007)]], []]',
    '"0N06MRuuX5WQjOvyYDy0O$"': '[[[(25.7248227743324, 21.823244124007), (21.8248227743324, 21.823244124007), (21.8248227743324, 22.073244124007), (25.7248227743324, 22.073244124007)]], []]',
    '"0N06MRuuX5WQjOvyYDy0O_"': '[[[(25.7248227743324, 22.073244124007), (25.4748227743324, 22.073244124007), (25.4748227743324, 24.673244124007), (25.7248227743324, 24.673244124007)]], []]',
    '"0N06MRuuX5WQjOvyYDy0OY"': '[[[(14.9511794330828, 33.5072879596117), (14.6511794330828, 33.5072879596117), (14.6511794330828, 34.0072879596117), (14.9511794330828, 34.0072879596117)]], []]',
    '"0N06MRuuX5WQjOvyYDy0Oa"': '[[[(32.6511794330828, 33.5072879596117), (32.3511794330828, 33.5072879596117), (32.3511794330828, 34.0072879596117), (32.6511794330828, 34.0072879596117)]], []]',
    '"0N06MRuuX5WQjOvyYDy0Oc"': '[[[(32.6511794330828, 12.0072879596116), (32.3511794330828, 12.0072879596116), (32.3511794330828, 12.5072879596116), (32.6511794330828, 12.5072879596116)]], []]',
    '"0N06MRuuX5WQjOvyYDy0Oe"': '[[[(23.9011794330828, 12.0072879596117), (23.4011794330828, 12.0072879596117), (23.4011794330828, 12.3072879596117), (23.9011794330828, 12.3072879596117)]], []]',
    '"0N06MRuuX5WQjOvyYDy0Og"': '[[[(14.9511794330828, 12.0072879596117), (14.6511794330828, 12.0072879596117), (14.6511794330828, 12.5072879596117), (14.9511794330828, 12.5072879596117)]], []]',
    '"0N06MRuuX5WQjOvyYDy0Oi"': '[[[(23.9011794330829, 33.7072879596117), (23.4011794330829, 33.7072879596117), (23.4011794330829, 34.0072879596117), (23.9011794330829, 34.0072879596117)]], []]',
    '"0N06MRuuX5WQjOvyYDy0Ok"': '[[[(14.9511794330828, 28.2124985896816), (14.6511794330828, 28.2124985896816), (14.6511794330828, 28.7124985896816), (14.9511794330828, 28.7124985896816)]], []]',
    '"0N06MRuuX5WQjOvyYDy0OG"': '[[[(32.6511794330828, 28.2124985896815), (32.3511794330828, 28.2124985896815), (32.3511794330828, 28.7124985896815), (32.6511794330828, 28.7124985896815)]], []]',
    '"0N06MRuuX5WQjOvyYDy0OI"': '[[[(14.9511794330828, 17.2572879596116), (14.6511794330828, 17.2572879596116), (14.6511794330828, 17.7572879596116), (14.9511794330828, 17.7572879596116)]], []]',
    '"0N06MRuuX5WQjOvyYDy0OK"': '[[[(32.6511794330828, 17.2572879596116), (32.3511794330828, 17.2572879596116), (32.3511794330828, 17.7572879596116), (32.6511794330828, 17.7572879596116)]], []]',
    '"0N06MRuuX5WQjOvyYDy0OM"': '[[[(32.6511794330828, 22.7572879596116), (32.3511794330828, 22.7572879596116), (32.3511794330828, 23.2572879596116), (32.6511794330828, 23.2572879596116)]], []]',
    '"0N06MRuuX5WQjOvyYDy0OO"': '[[[(14.9511794330828, 22.7572879596117), (14.6511794330828, 22.7572879596117), (14.6511794330828, 23.2572879596117), (14.9511794330828, 23.2572879596117)]], []]',
    '"0N06MRuuX5WQjOvzkDy0OX"': '[[[(25.0748227743324, 20.549244124007), (24.2748227743324, 20.549244124007), (24.2748227743324, 23.597244124007), (25.0748227743324, 23.597244124007)]], []]',
    '"0N06MRuuX5WQjOvyYDy0OQ"': '[[[(19.5261794330828, 12.0072879596117), (19.0261794330828, 12.0072879596117), (19.0261794330828, 12.3072879596117), (19.5261794330828, 12.3072879596117)]], []]',
    '"0N06MRuuX5WQjOvyYDy0OS"': '[[[(28.5261794330828, 12.0072879596116), (28.0261794330828, 12.0072879596116), (28.0261794330828, 12.3072879596116), (28.5261794330828, 12.3072879596116)]], []]',
    '"0N06MRuuX5WQjOvyYDy0OU"': '[[[(19.5261794330828, 33.7072879596117), (19.0261794330828, 33.7072879596117), (19.0261794330828, 34.0072879596117), (19.5261794330828, 34.0072879596117)]], []]',
    '"0N06MRuuX5WQjOvyYDy0O0"': '[[[(28.5261794330828, 33.7072879596117), (28.0261794330828, 33.7072879596117), (28.0261794330828, 34.0072879596117), (28.5261794330828, 34.0072879596117)]], []]',
    '"0N06MRuuX5WQjOvyYDy0O4"': '[[[(23.4011794330829, 24.673244124007), (21.5748227743324, 24.673244124007), (21.5748227743324, 22.7572879596117), (14.6511794330828, 22.7572879596117), (14.6511794330828, 34.0072879596117), (23.4011794330829, 34.0072879596117)]], []]',
    '"0N06MRuuX5WQjOvyYDy0OF"': '[[[(32.6511794330828, 23.2572879596117), (25.7248227743324, 23.2572879596117), (25.7248227743324, 24.673244124007), (23.4011794330829, 24.673244124007), (23.4011794330829, 34.0072879596117), (32.6511794330828, 34.0072879596117)]], []]',
    '"0N06MRuuX5WQjOvyYDy0Ps"': '[[[(21.5748227743324, 21.823244124007), (23.9011794330828, 21.823244124007), (23.9011794330828, 12.0072879596117), (14.6511794330828, 12.0072879596117), (14.6511794330828, 22.7572879596117), (21.5748227743324, 22.7572879596117)]], []]',
    '"0N06MRuuX5WQjOvyYDy0PX"': '[[[(32.6511794330829, 12.0072879596116), (23.9011794330829, 12.0072879596116), (23.9011794330829, 21.823244124007), (25.7248227743324, 21.823244124007), (25.7248227743324, 23.2572879596116), (32.6511794330829, 23.2572879596116)]], []]',
    '"0N06MRuuX5WQjOvyYDy0MJ"': '[[[(25.4748227743324, 24.423244124007), (21.5748227743324, 24.423244124007), (21.5748227743324, 24.673244124007), (25.4748227743324, 24.673244124007)]], []]',
    '"0N06MRuuX5WQjOvyYDy0MI"': '[[[(21.8248227743324, 21.823244124007), (21.5748227743324, 21.823244124007), (21.5748227743324, 24.423244124007), (21.8248227743324, 24.423244124007)]], []]',
    '"0N06MRuuX5WQjOvyYDy0ML"': '[[[(25.7248227743324, 21.823244124007), (21.8248227743324, 21.823244124007), (21.8248227743324, 22.073244124007), (25.7248227743324, 22.073244124007)]], []]',
    '"0N06MRuuX5WQjOvyYDy0MK"': '[[[(25.7248227743324, 22.073244124007), (25.4748227743324, 22.073244124007), (25.4748227743324, 24.673244124007), (25.7248227743324, 24.673244124007)]], []]',
    '"0N06MRuuX5WQjOvyYDy0MO"': '[[[(14.9511794330828, 33.5072879596117), (14.6511794330828, 33.5072879596117), (14.6511794330828, 34.0072879596117), (14.9511794330828, 34.0072879596117)]], []]',
    '"0N06MRuuX5WQjOvyYDy0MQ"': '[[[(32.6511794330828, 33.5072879596117), (32.3511794330828, 33.5072879596117), (32.3511794330828, 34.0072879596117), (32.6511794330828, 34.0072879596117)]], []]',
    '"0N06MRuuX5WQjOvyYDy0MS"': '[[[(32.6511794330828, 12.0072879596116), (32.3511794330828, 12.0072879596116), (32.3511794330828, 12.5072879596116), (32.6511794330828, 12.5072879596116)]], []]',
    '"0N06MRuuX5WQjOvyYDy0MU"': '[[[(23.9011794330828, 12.0072879596117), (23.4011794330828, 12.0072879596117), (23.4011794330828, 12.3072879596117), (23.9011794330828, 12.3072879596117)]], []]',
    '"0N06MRuuX5WQjOvyYDy0M0"': '[[[(14.9511794330828, 12.0072879596117), (14.6511794330828, 12.0072879596117), (14.6511794330828, 12.5072879596117), (14.9511794330828, 12.5072879596117)]], []]',
    '"0N06MRuuX5WQjOvyYDy0M2"': '[[[(23.9011794330829, 33.7072879596117), (23.4011794330829, 33.7072879596117), (23.4011794330829, 34.0072879596117), (23.9011794330829, 34.0072879596117)]], []]',
    '"0N06MRuuX5WQjOvyYDy0M4"': '[[[(14.9511794330828, 28.2124985896816), (14.6511794330828, 28.2124985896816), (14.6511794330828, 28.7124985896816), (14.9511794330828, 28.7124985896816)]], []]',
    '"0N06MRuuX5WQjOvyYDy0M6"': '[[[(32.6511794330828, 28.2124985896815), (32.3511794330828, 28.2124985896815), (32.3511794330828, 28.7124985896815), (32.6511794330828, 28.7124985896815)]], []]',
    '"0N06MRuuX5WQjOvyYDy0M8"': '[[[(14.9511794330828, 17.2572879596116), (14.6511794330828, 17.2572879596116), (14.6511794330828, 17.7572879596116), (14.9511794330828, 17.7572879596116)]], []]',
    '"0N06MRuuX5WQjOvyYDy0MA"': '[[[(32.6511794330828, 17.2572879596116), (32.3511794330828, 17.2572879596116), (32.3511794330828, 17.7572879596116), (32.6511794330828, 17.7572879596116)]], []]',
    '"0N06MRuuX5WQjOvyYDy0MC"': '[[[(32.6511794330828, 22.7572879596116), (32.3511794330828, 22.7572879596116), (32.3511794330828, 23.2572879596116), (32.6511794330828, 23.2572879596116)]], []]',
    '"0N06MRuuX5WQjOvyYDy0ME"': '[[[(14.9511794330828, 22.7572879596117), (14.6511794330828, 22.7572879596117), (14.6511794330828, 23.2572879596117), (14.9511794330828, 23.2572879596117)]], []]',
    '"0N06MRuuX5WQjOvyYDy0Nm"': '[[[(19.5261794330828, 12.0072879596117), (19.0261794330828, 12.0072879596117), (19.0261794330828, 12.3072879596117), (19.5261794330828, 12.3072879596117)]], []]',
    '"0N06MRuuX5WQjOvzkDy0OW"': '[[[(25.0748227743324, 22.899244124007), (24.2748227743324, 22.899244124007), (24.2748227743324, 25.947244124007), (25.0748227743324, 25.947244124007)]], []]',
    '"0N06MRuuX5WQjOvyYDy0No"': '[[[(28.5261794330828, 12.0072879596116), (28.0261794330828, 12.0072879596116), (28.0261794330828, 12.3072879596116), (28.5261794330828, 12.3072879596116)]], []]',
    '"0N06MRuuX5WQjOvyYDy0Nq"': '[[[(19.5261794330828, 33.7072879596117), (19.0261794330828, 33.7072879596117), (19.0261794330828, 34.0072879596117), (19.5261794330828, 34.0072879596117)]], []]',
    '"0N06MRuuX5WQjOvyYDy0Ns"': '[[[(28.5261794330828, 33.7072879596117), (28.0261794330828, 33.7072879596117), (28.0261794330828, 34.0072879596117), (28.5261794330828, 34.0072879596117)]], []]',
    '"0N06MRuuX5WQjOvyYDy0Nw"': '[[[(23.4011794330829, 24.673244124007), (21.5748227743324, 24.673244124007), (21.5748227743324, 22.7572879596117), (14.6511794330828, 22.7572879596117), (14.6511794330828, 34.0072879596117), (23.4011794330829, 34.0072879596117)]], []]',
    '"0N06MRuuX5WQjOvyYDy0Nb"': '[[[(32.6511794330828, 23.2572879596117), (25.7248227743324, 23.2572879596117), (25.7248227743324, 24.673244124007), (23.4011794330829, 24.673244124007), (23.4011794330829, 34.0072879596117), (32.6511794330828, 34.0072879596117)]], []]',
    '"0N06MRuuX5WQjOvyYDy0Ni"': '[[[(21.5748227743324, 21.823244124007), (23.9011794330828, 21.823244124007), (23.9011794330828, 12.0072879596117), (14.6511794330828, 12.0072879596117), (14.6511794330828, 22.7572879596117), (21.5748227743324, 22.7572879596117)]], []]',
    '"0N06MRuuX5WQjOvyYDy0NN"': '[[[(32.6511794330829, 12.0072879596116), (23.9011794330829, 12.0072879596116), (23.9011794330829, 21.823244124007), (25.7248227743324, 21.823244124007), (25.7248227743324, 23.2572879596116), (32.6511794330829, 23.2572879596116)]], []]',
    '"0N06MRuuX5WQjOvyYDy3lQ"': '[[[(25.4748227743324, 24.423244124007), (21.5748227743324, 24.423244124007), (21.5748227743324, 24.673244124007), (25.4748227743324, 24.673244124007)]], []]',
    '"0N06MRuuX5WQjOvyYDy3lT"': '[[[(21.8248227743324, 21.823244124007), (21.5748227743324, 21.823244124007), (21.5748227743324, 24.423244124007), (21.8248227743324, 24.423244124007)]], []]',
    '"0N06MRuuX5WQjOvyYDy3lS"': '[[[(25.7248227743324, 21.823244124007), (21.8248227743324, 21.823244124007), (21.8248227743324, 22.073244124007), (25.7248227743324, 22.073244124007)]], []]',
    '"0N06MRuuX5WQjOvyYDy3lV"': '[[[(25.7248227743324, 22.073244124007), (25.4748227743324, 22.073244124007), (25.4748227743324, 24.673244124007), (25.7248227743324, 24.673244124007)]], []]',
    '"0N06MRuuX5WQjOvyYDy3l3"': '[[[(14.9511794330828, 33.5072879596117), (14.6511794330828, 33.5072879596117), (14.6511794330828, 34.0072879596117), (14.9511794330828, 34.0072879596117)]], []]',
    '"0N06MRuuX5WQjOvyYDy3l5"': '[[[(32.6511794330828, 33.5072879596117), (32.3511794330828, 33.5072879596117), (32.3511794330828, 34.0072879596117), (32.6511794330828, 34.0072879596117)]], []]',
    '"0N06MRuuX5WQjOvyYDy3l7"': '[[[(32.6511794330828, 12.0072879596116), (32.3511794330828, 12.0072879596116), (32.3511794330828, 12.5072879596116), (32.6511794330828, 12.5072879596116)]], []]',
    '"0N06MRuuX5WQjOvyYDy3l9"': '[[[(23.9011794330828, 12.0072879596117), (23.4011794330828, 12.0072879596117), (23.4011794330828, 12.3072879596117), (23.9011794330828, 12.3072879596117)]], []]',
    '"0N06MRuuX5WQjOvyYDy3lB"': '[[[(14.9511794330828, 12.0072879596117), (14.6511794330828, 12.0072879596117), (14.6511794330828, 12.5072879596117), (14.9511794330828, 12.5072879596117)]], []]',
    '"0N06MRuuX5WQjOvyYDy3lD"': '[[[(23.9011794330829, 33.7072879596117), (23.4011794330829, 33.7072879596117), (23.4011794330829, 34.0072879596117), (23.9011794330829, 34.0072879596117)]], []]',
    '"0N06MRuuX5WQjOvyYDy3lF"': '[[[(14.9511794330828, 28.2124985896816), (14.6511794330828, 28.2124985896816), (14.6511794330828, 28.7124985896816), (14.9511794330828, 28.7124985896816)]], []]',
    '"0N06MRuuX5WQjOvyYDy3in"': '[[[(32.6511794330828, 28.2124985896815), (32.3511794330828, 28.2124985896815), (32.3511794330828, 28.7124985896815), (32.6511794330828, 28.7124985896815)]], []]',
    '"0N06MRuuX5WQjOvyYDy3ip"': '[[[(14.9511794330828, 17.2572879596116), (14.6511794330828, 17.2572879596116), (14.6511794330828, 17.7572879596116), (14.9511794330828, 17.7572879596116)]], []]',
    '"0N06MRuuX5WQjOvyYDy3ir"': '[[[(32.6511794330828, 17.2572879596116), (32.3511794330828, 17.2572879596116), (32.3511794330828, 17.7572879596116), (32.6511794330828, 17.7572879596116)]], []]',
    '"0N06MRuuX5WQjOvyYDy3it"': '[[[(32.6511794330828, 22.7572879596116), (32.3511794330828, 22.7572879596116), (32.3511794330828, 23.2572879596116), (32.6511794330828, 23.2572879596116)]], []]',
    '"0N06MRuuX5WQjOvyYDy3iv"': '[[[(14.9511794330828, 22.7572879596117), (14.6511794330828, 22.7572879596117), (14.6511794330828, 23.2572879596117), (14.9511794330828, 23.2572879596117)]], []]',
    '"0N06MRuuX5WQjOvyYDy3ix"': '[[[(19.5261794330828, 12.0072879596117), (19.0261794330828, 12.0072879596117), (19.0261794330828, 12.3072879596117), (19.5261794330828, 12.3072879596117)]], []]',
    '"0N06MRuuX5WQjOvyYDy3iz"': '[[[(28.5261794330828, 12.0072879596116), (28.0261794330828, 12.0072879596116), (28.0261794330828, 12.3072879596116), (28.5261794330828, 12.3072879596116)]], []]',
    '"0N06MRuuX5WQjOvzkDy0TC"': '[[[(23.0248227743324, 20.549244124007), (22.2248227743324, 20.549244124007), (22.2248227743324, 23.597244124007), (23.0248227743324, 23.597244124007)]], []]',
    '"0N06MRuuX5WQjOvyYDy3i$"': '[[[(19.5261794330828, 33.7072879596117), (19.0261794330828, 33.7072879596117), (19.0261794330828, 34.0072879596117), (19.5261794330828, 34.0072879596117)]], []]',
    '"0N06MRuuX5WQjOvyYDy3iX"': '[[[(28.5261794330828, 33.7072879596117), (28.0261794330828, 33.7072879596117), (28.0261794330828, 34.0072879596117), (28.5261794330828, 34.0072879596117)]], []]',
    '"0N06MRuuX5WQjOvyYDy3ib"': '[[[(23.4011794330829, 24.673244124007), (21.5748227743324, 24.673244124007), (21.5748227743324, 22.7572879596117), (14.6511794330828, 22.7572879596117), (14.6511794330828, 34.0072879596117), (23.4011794330829, 34.0072879596117)]], []]',
    '"0N06MRuuX5WQjOvyYDy3ii"': '[[[(32.6511794330828, 23.2572879596117), (25.7248227743324, 23.2572879596117), (25.7248227743324, 24.673244124007), (23.4011794330829, 24.673244124007), (23.4011794330829, 34.0072879596117), (32.6511794330828, 34.0072879596117)]], []]',
    '"0N06MRuuX5WQjOvyYDy3iN"': '[[[(21.5748227743324, 21.823244124007), (23.9011794330828, 21.823244124007), (23.9011794330828, 12.0072879596117), (14.6511794330828, 12.0072879596117), (14.6511794330828, 22.7572879596117), (21.5748227743324, 22.7572879596117)]], []]',
    '"0N06MRuuX5WQjOvyYDy3iU"': '[[[(32.6511794330829, 12.0072879596116), (23.9011794330829, 12.0072879596116), (23.9011794330829, 21.823244124007), (25.7248227743324, 21.823244124007), (25.7248227743324, 23.2572879596116), (32.6511794330829, 23.2572879596116)]], []]',
    '"0N06MRuuX5WQjOvyYDy3iE"': '[[[(25.7248227743324, 21.823244124007), (21.5748227743324, 21.823244124007), (21.5748227743324, 24.673244124007), (25.7248227743324, 24.673244124007)]], []]',
    '"0A8QdCd812$xV$VdhPItzP"': '[[[(45.4945091551814, 3.4318681745907), (2.4945091551814, 3.4318681745907), (2.4945091551814, 44.4318681745907), (45.4945091551814, 44.4318681745907)]], [[(32.6542973762976, 34.0150569786985), (14.6542973762976, 34.0150569786985), (14.6542973762976, 12.0150569786984), (32.6542973762976, 12.0150569786984)]]]',
    '"0A8QdCd812$xV$VdhPItzI"': '[[[(32.6542973762976, 12.0150569786984), (14.6542973762976, 12.0150569786984), (14.6542973762976, 34.0150569786985), (32.6542973762976, 34.0150569786985)]], []]',
    '"3eORFXfAH04e_QSxfI$ODY"': '[[[(25.0748227743324, 22.899244124007), (24.2748227743324, 22.899244124007), (24.2748227743324, 25.947244124007), (25.0748227743324, 25.947244124007)]], []]',
    '"3eORFXfAH04e_QSxfI$ODX"': '[[[(23.0248227743324, 20.549244124007), (22.2248227743324, 20.549244124007), (22.2248227743324, 23.597244124007), (23.0248227743324, 23.597244124007)]], []]',
    '"3eORFXfAH04e_QSxfI$ODZ"': '[[[(25.0748227743324, 20.549244124007), (24.2748227743324, 20.549244124007), (24.2748227743324, 23.597244124007), (25.0748227743324, 23.597244124007)]], []]',
    '"3eORFXfAH04e_QSxfI$O1C"': '[[[(25.0748227743324, 22.899244124007), (24.2748227743324, 22.899244124007), (24.2748227743324, 25.947244124007), (25.0748227743324, 25.947244124007)]], []]',
    '"3eORFXfAH04e_QSxfI$O06"': '[[[(25.0748227743324, 20.549244124007), (24.2748227743324, 20.549244124007), (24.2748227743324, 23.597244124007), (25.0748227743324, 23.597244124007)]], []]',
    '"3eORFXfAH04e_QSxfI$O9p"': '[[[(23.0248227743324, 20.549244124007), (22.2248227743324, 20.549244124007), (22.2248227743324, 23.597244124007), (23.0248227743324, 23.597244124007)]], []]',
    '"3eORFXfAH04e_QSxfI$ODp"': '[[[(25.0748227743324, 22.899244124007), (24.2748227743324, 22.899244124007), (24.2748227743324, 25.947244124007), (25.0748227743324, 25.947244124007)]], []]',
    '"3eORFXfAH04e_QSxfI$ODq"': '[[[(25.0748227743324, 20.549244124007), (24.2748227743324, 20.549244124007), (24.2748227743324, 23.597244124007), (25.0748227743324, 23.597244124007)]], []]',
    '"3eORFXfAH04e_QSxfI$ODo"': '[[[(23.0248227743324, 20.549244124007), (22.2248227743324, 20.549244124007), (22.2248227743324, 23.597244124007), (23.0248227743324, 23.597244124007)]], []]',
    '"3eORFXfAH04e_QSxfI$OD1"': '[[[(25.0748227743324, 22.899244124007), (24.2748227743324, 22.899244124007), (24.2748227743324, 25.947244124007), (25.0748227743324, 25.947244124007)]], []]',
    '"3eORFXfAH04e_QSxfI$OD2"': '[[[(25.0748227743324, 20.549244124007), (24.2748227743324, 20.549244124007), (24.2748227743324, 23.597244124007), (25.0748227743324, 23.597244124007)]], []]',
    '"3eORFXfAH04e_QSxfI$OD0"': '[[[(23.0248227743324, 20.549244124007), (22.2248227743324, 20.549244124007), (22.2248227743324, 23.597244124007), (23.0248227743324, 23.597244124007)]], []]',
    '"3eORFXfAH04e_QSxfI$ODG"': '[[[(25.0748227743324, 22.899244124007), (24.2748227743324, 22.899244124007), (24.2748227743324, 25.947244124007), (25.0748227743324, 25.947244124007)]], []]',
    '"3eORFXfAH04e_QSxfI$ODH"': '[[[(25.0748227743324, 20.549244124007), (24.2748227743324, 20.549244124007), (24.2748227743324, 23.597244124007), (25.0748227743324, 23.597244124007)]], []]',
    '"3eORFXfAH04e_QSxfI$ODV"': '[[[(23.0248227743324, 20.549244124007), (22.2248227743324, 20.549244124007), (22.2248227743324, 23.597244124007), (23.0248227743324, 23.597244124007)]], []]',
    '"0N06MRuuX5WQjOvzkDy0TA"': '[[[(25.0748227743324, 20.549244124007), (24.2748227743324, 20.549244124007), (24.2748227743324, 23.597244124007), (25.0748227743324, 23.597244124007)]], []]',
    '"3eORFXfAH04e_QSxfI$OIW"': '[[[(25.0748227743324, 22.899244124007), (24.2748227743324, 22.899244124007), (24.2748227743324, 25.947244124007), (25.0748227743324, 25.947244124007)]], []]',
    '"3eORFXfAH04e_QSxfI$OIX"': '[[[(25.0748227743324, 20.549244124007), (24.2748227743324, 20.549244124007), (24.2748227743324, 23.597244124007), (25.0748227743324, 23.597244124007)]], []]',
    '"3eORFXfAH04e_QSxfI$OIl"': '[[[(23.0248227743324, 20.549244124007), (22.2248227743324, 20.549244124007), (22.2248227743324, 23.597244124007), (23.0248227743324, 23.597244124007)]], []]',
    '"0N06MRuuX5WQjOvzkDy0TD"': '[[[(25.0748227743324, 22.899244124007), (24.2748227743324, 22.899244124007), (24.2748227743324, 25.947244124007), (25.0748227743324, 25.947244124007)]], []]'}

In [250]:
data = """

voids(product("0N06MRuuX5WQjOvzkDy3l0"), product("0N06MRuuX5WQjOvyYDy3lS")).
voids(product("0N06MRuuX5WQjOvzkDy3lU"), product("0N06MRuuX5WQjOvyYDy3lS")).
voids(product("0N06MRuuX5WQjOvzkDy3l1"), product("0N06MRuuX5WQjOvyYDy3lQ")).
voids(product("0N06MRuuX5WQjOvzkDy0TD"), product("0N06MRuuX5WQjOvyYDy0T6")).
voids(product("0N06MRuuX5WQjOvzkDy0MP"), product("0N06MRuuX5WQjOvyYDy0ML")).
voids(product("0N06MRuuX5WQjOvzkDy0MN"), product("0N06MRuuX5WQjOvyYDy0ML")).
voids(product("0N06MRuuX5WQjOvzkDy0MM"), product("0N06MRuuX5WQjOvyYDy0MJ")).
voids(product("0N06MRuuX5WQjOvzkDy0OZ"), product("0N06MRuuX5WQjOvyYDy0O$")).
voids(product("0N06MRuuX5WQjOvzkDy0OX"), product("0N06MRuuX5WQjOvyYDy0O$")).
voids(product("0N06MRuuX5WQjOvzkDy0OW"), product("0N06MRuuX5WQjOvyYDy0Oz")).
voids(product("0N06MRuuX5WQjOvzkDy0TC"), product("0N06MRuuX5WQjOvyYDy0T8")).
voids(product("3eORFXfAH04e_QSxfI$ODY"), product("3eHkYig055Ex7oFuhZVIqN")).
voids(product("3eORFXfAH04e_QSxfI$ODX"), product("3eHkYig055Ex7oFuhZVIqf")).
voids(product("3eORFXfAH04e_QSxfI$ODZ"), product("3eHkYig055Ex7oFuhZVIqf")).
voids(product("3eORFXfAH04e_QSxfI$O1C"), product("3eHkYig055Ex7oFuhZVInm")).
voids(product("3eORFXfAH04e_QSxfI$O06"), product("3eHkYig055Ex7oFuhZVIno")).
voids(product("3eORFXfAH04e_QSxfI$O9p"), product("3eHkYig055Ex7oFuhZVIno")).
voids(product("3eORFXfAH04e_QSxfI$ODp"), product("3eHkYig055Ex7oFuhZVI_Q")).
voids(product("3eORFXfAH04e_QSxfI$ODq"), product("3eHkYig055Ex7oFuhZVI_S")).
voids(product("3eORFXfAH04e_QSxfI$ODo"), product("3eHkYig055Ex7oFuhZVI_S")).
voids(product("3eORFXfAH04e_QSxfI$OD1"), product("3eHkYig055Ex7oFuhZVIya")).
voids(product("0N06MRuuX5WQjOvzkDy0TA"), product("0N06MRuuX5WQjOvyYDy0T8")).
voids(product("3eORFXfAH04e_QSxfI$OD2"), product("3eHkYig055Ex7oFuhZVIyc")).
voids(product("3eORFXfAH04e_QSxfI$OD0"), product("3eHkYig055Ex7oFuhZVIyc")).
voids(product("3eORFXfAH04e_QSxfI$ODG"), product("3eHkYig055Ex7oFuhZVIvF")).
voids(product("3eORFXfAH04e_QSxfI$ODH"), product("3eHkYig055Ex7oFuhZVIv1")).
voids(product("3eORFXfAH04e_QSxfI$ODV"), product("3eHkYig055Ex7oFuhZVIv1")).
voids(product("3eORFXfAH04e_QSxfI$OIW"), product("3eHkYig055Ex7oFuhZVId2")).
voids(product("3eORFXfAH04e_QSxfI$OIX"), product("3eHkYig055Ex7oFuhZVId4")).
voids(product("3eORFXfAH04e_QSxfI$OIl"), product("3eHkYig055Ex7oFuhZVId4")).


is_a(product("3eHkYig055Ex7oFuhZVTBc"), slab).
containedIn(product("3eHkYig055Ex7oFuhZVTBc"), storey(2)).
is_a(product("3eHkYig055Ex7oFuhZVTAV"), slab).
containedIn(product("3eHkYig055Ex7oFuhZVTAV"), storey(2)).
is_a(product("3eHkYig055Ex7oFuhZVTAb"), slab).
containedIn(product("3eHkYig055Ex7oFuhZVTAb"), storey(2)).
is_a(product("3eHkYig055Ex7oFuhZVT98"), slab).
containedIn(product("3eHkYig055Ex7oFuhZVT98"), storey(2)).
is_a(product("3eHkYig055Ex7oFuhZVIqN"), wall).
containedIn(product("3eHkYig055Ex7oFuhZVIqN"), storey(1)).
is_a(product("3eHkYig055Ex7oFuhZVIqM"), wall).
containedIn(product("3eHkYig055Ex7oFuhZVIqM"), storey(1)).
is_a(product("3eHkYig055Ex7oFuhZVIqf"), wall).
containedIn(product("3eHkYig055Ex7oFuhZVIqf"), storey(1)).
is_a(product("3eHkYig055Ex7oFuhZVIqe"), wall).
containedIn(product("3eHkYig055Ex7oFuhZVIqe"), storey(1)).
is_a(product("3eHkYig055Ex7oFuhZVInm"), wall).
containedIn(product("3eHkYig055Ex7oFuhZVInm"), storey(2)).
is_a(product("3eHkYig055Ex7oFuhZVInp"), wall).
containedIn(product("3eHkYig055Ex7oFuhZVInp"), storey(2)).
is_a(product("3eHkYig055Ex7oFuhZVIno"), wall).
containedIn(product("3eHkYig055Ex7oFuhZVIno"), storey(2)).
is_a(product("3eHkYig055Ex7oFuhZVInr"), wall).
containedIn(product("3eHkYig055Ex7oFuhZVInr"), storey(2)).
is_a(product("3eHkYig055Ex7oFuhZVI_Q"), wall).
containedIn(product("3eHkYig055Ex7oFuhZVI_Q"), storey(3)).
is_a(product("3eHkYig055Ex7oFuhZVI_T"), wall).
containedIn(product("3eHkYig055Ex7oFuhZVI_T"), storey(3)).
is_a(product("3eHkYig055Ex7oFuhZVI_S"), wall).
containedIn(product("3eHkYig055Ex7oFuhZVI_S"), storey(3)).
is_a(product("3eHkYig055Ex7oFuhZVI_V"), wall).
containedIn(product("3eHkYig055Ex7oFuhZVI_V"), storey(3)).
is_a(product("3eHkYig055Ex7oFuhZVIya"), wall).
containedIn(product("3eHkYig055Ex7oFuhZVIya"), storey(4)).
is_a(product("3eHkYig055Ex7oFuhZVIyd"), wall).
containedIn(product("3eHkYig055Ex7oFuhZVIyd"), storey(4)).
is_a(product("3eHkYig055Ex7oFuhZVIyc"), wall).
containedIn(product("3eHkYig055Ex7oFuhZVIyc"), storey(4)).
is_a(product("3eHkYig055Ex7oFuhZVIyv"), wall).
containedIn(product("3eHkYig055Ex7oFuhZVIyv"), storey(4)).
is_a(product("3eHkYig055Ex7oFuhZVIvF"), wall).
containedIn(product("3eHkYig055Ex7oFuhZVIvF"), storey(5)).
is_a(product("3eHkYig055Ex7oFuhZVIvE"), wall).
containedIn(product("3eHkYig055Ex7oFuhZVIvE"), storey(5)).
is_a(product("3eHkYig055Ex7oFuhZVIv1"), wall).
containedIn(product("3eHkYig055Ex7oFuhZVIv1"), storey(5)).
is_a(product("3eHkYig055Ex7oFuhZVIv0"), wall).
containedIn(product("3eHkYig055Ex7oFuhZVIv0"), storey(5)).
is_a(product("3eHkYig055Ex7oFuhZVId2"), wall).
containedIn(product("3eHkYig055Ex7oFuhZVId2"), storey(6)).
is_a(product("3eHkYig055Ex7oFuhZVId5"), wall).
containedIn(product("3eHkYig055Ex7oFuhZVId5"), storey(6)).
is_a(product("3eHkYig055Ex7oFuhZVId4"), wall).
containedIn(product("3eHkYig055Ex7oFuhZVId4"), storey(6)).
is_a(product("3eHkYig055Ex7oFuhZVId7"), wall).
containedIn(product("3eHkYig055Ex7oFuhZVId7"), storey(6)).
is_a(product("3eHkYig055Ex7oFuhZVIZL"), slab).
containedIn(product("3eHkYig055Ex7oFuhZVIZL"), storey(1)).
is_a(product("3eHkYig055Ex7oFuhZVIYt"), slab).
containedIn(product("3eHkYig055Ex7oFuhZVIYt"), storey(1)).
is_a(product("3eHkYig055Ex7oFuhZVIW2"), slab).
containedIn(product("3eHkYig055Ex7oFuhZVIW2"), storey(1)).
is_a(product("2bxL$O5Cz3xOWd38LvRNOO"), slab).
containedIn(product("2bxL$O5Cz3xOWd38LvRNOO"), storey(1)).
is_a(product("2AICztFX95Z8QtA12iaPQo"), column).
containedIn(product("2AICztFX95Z8QtA12iaPQo"), storey(2)).
is_a(product("2AICztFX95Z8QtA12iaPQq"), column).
containedIn(product("2AICztFX95Z8QtA12iaPQq"), storey(2)).
is_a(product("2AICztFX95Z8QtA12iaPQs"), column).
containedIn(product("2AICztFX95Z8QtA12iaPQs"), storey(2)).
is_a(product("2AICztFX95Z8QtA12iaPR8"), column).
containedIn(product("2AICztFX95Z8QtA12iaPR8"), storey(2)).
is_a(product("2AICztFX95Z8QtA12iaPRA"), column).
containedIn(product("2AICztFX95Z8QtA12iaPRA"), storey(2)).
is_a(product("2AICztFX95Z8QtA12iaPRC"), column).
containedIn(product("2AICztFX95Z8QtA12iaPRC"), storey(2)).
is_a(product("2AICztFX95Z8QtA12iaPRE"), column).
containedIn(product("2AICztFX95Z8QtA12iaPRE"), storey(2)).
is_a(product("2AICztFX95Z8QtA12iaPR0"), column).
containedIn(product("2AICztFX95Z8QtA12iaPR0"), storey(2)).
is_a(product("2AICztFX95Z8QtA12iaPR2"), column).
containedIn(product("2AICztFX95Z8QtA12iaPR2"), storey(2)).
is_a(product("2AICztFX95Z8QtA12iaPR4"), column).
containedIn(product("2AICztFX95Z8QtA12iaPR4"), storey(2)).
is_a(product("2AICztFX95Z8QtA12iaPR6"), column).
containedIn(product("2AICztFX95Z8QtA12iaPR6"), storey(2)).
is_a(product("2AICztFX95Z8QtA12iaPRO"), column).
containedIn(product("2AICztFX95Z8QtA12iaPRO"), storey(2)).
is_a(product("3eORFXfAH04e_QSwbI$PvJ"), column).
containedIn(product("3eORFXfAH04e_QSwbI$PvJ"), storey(2)).
is_a(product("3eORFXfAH04e_QSwbI$P_Y"), column).
containedIn(product("3eORFXfAH04e_QSwbI$P_Y"), storey(2)).
is_a(product("3eORFXfAH04e_QSwbI$P$5"), column).
containedIn(product("3eORFXfAH04e_QSwbI$P$5"), storey(2)).
is_a(product("3eORFXfAH04e_QSwbI$P$3"), column).
containedIn(product("3eORFXfAH04e_QSwbI$P$3"), storey(2)).
is_a(product("02NiuWFer1lA4a2StaofHI"), column).
containedIn(product("02NiuWFer1lA4a2StaofHI"), storey(3)).
is_a(product("02NiuWFer1lA4a2StaofHG"), column).
containedIn(product("02NiuWFer1lA4a2StaofHG"), storey(3)).
is_a(product("02NiuWFer1lA4a2StaofHM"), column).
containedIn(product("02NiuWFer1lA4a2StaofHM"), storey(3)).
is_a(product("02NiuWFer1lA4a2StaofHK"), column).
containedIn(product("02NiuWFer1lA4a2StaofHK"), storey(3)).
is_a(product("02NiuWFer1lA4a2StaofIg"), column).
containedIn(product("02NiuWFer1lA4a2StaofIg"), storey(3)).
is_a(product("02NiuWFer1lA4a2StaofIe"), column).
containedIn(product("02NiuWFer1lA4a2StaofIe"), storey(3)).
is_a(product("02NiuWFer1lA4a2StaofIk"), column).
containedIn(product("02NiuWFer1lA4a2StaofIk"), storey(3)).
is_a(product("02NiuWFer1lA4a2StaofIi"), column).
containedIn(product("02NiuWFer1lA4a2StaofIi"), storey(3)).
is_a(product("02NiuWFer1lA4a2StaofIY"), column).
containedIn(product("02NiuWFer1lA4a2StaofIY"), storey(3)).
is_a(product("02NiuWFer1lA4a2StaofIW"), column).
containedIn(product("02NiuWFer1lA4a2StaofIW"), storey(3)).
is_a(product("02NiuWFer1lA4a2StaofIc"), column).
containedIn(product("02NiuWFer1lA4a2StaofIc"), storey(3)).
is_a(product("02NiuWFer1lA4a2StaofIa"), column).
containedIn(product("02NiuWFer1lA4a2StaofIa"), storey(3)).
is_a(product("02NiuWFer1lA4a2StaofIw"), column).
containedIn(product("02NiuWFer1lA4a2StaofIw"), storey(3)).
is_a(product("02NiuWFer1lA4a2StaofIu"), column).
containedIn(product("02NiuWFer1lA4a2StaofIu"), storey(3)).
is_a(product("02NiuWFer1lA4a2StaofI_"), column).
containedIn(product("02NiuWFer1lA4a2StaofI_"), storey(3)).
is_a(product("02NiuWFer1lA4a2StaofIy"), column).
containedIn(product("02NiuWFer1lA4a2StaofIy"), storey(3)).
is_a(product("02NiuWFer1lA4a2StaofJx"), column).
containedIn(product("02NiuWFer1lA4a2StaofJx"), storey(4)).
is_a(product("02NiuWFer1lA4a2StaofJv"), column).
containedIn(product("02NiuWFer1lA4a2StaofJv"), storey(4)).
is_a(product("02NiuWFer1lA4a2StaofJ$"), column).
containedIn(product("02NiuWFer1lA4a2StaofJ$"), storey(4)).
is_a(product("02NiuWFer1lA4a2StaofJz"), column).
containedIn(product("02NiuWFer1lA4a2StaofJz"), storey(4)).
is_a(product("02NiuWFer1lA4a2StaofJp"), column).
containedIn(product("02NiuWFer1lA4a2StaofJp"), storey(4)).
is_a(product("02NiuWFer1lA4a2StaofJn"), column).
containedIn(product("02NiuWFer1lA4a2StaofJn"), storey(4)).
is_a(product("02NiuWFer1lA4a2StaofJt"), column).
containedIn(product("02NiuWFer1lA4a2StaofJt"), storey(4)).
is_a(product("02NiuWFer1lA4a2StaofJr"), column).
containedIn(product("02NiuWFer1lA4a2StaofJr"), storey(4)).
is_a(product("02NiuWFer1lA4a2StaofJB"), column).
containedIn(product("02NiuWFer1lA4a2StaofJB"), storey(4)).
is_a(product("02NiuWFer1lA4a2StaofJ9"), column).
containedIn(product("02NiuWFer1lA4a2StaofJ9"), storey(4)).
is_a(product("02NiuWFer1lA4a2StaofJF"), column).
containedIn(product("02NiuWFer1lA4a2StaofJF"), storey(4)).
is_a(product("02NiuWFer1lA4a2StaofJD"), column).
containedIn(product("02NiuWFer1lA4a2StaofJD"), storey(4)).
is_a(product("02NiuWFer1lA4a2StaofJ3"), column).
containedIn(product("02NiuWFer1lA4a2StaofJ3"), storey(4)).
is_a(product("02NiuWFer1lA4a2StaofJ1"), column).
containedIn(product("02NiuWFer1lA4a2StaofJ1"), storey(4)).
is_a(product("02NiuWFer1lA4a2StaofJ7"), column).
containedIn(product("02NiuWFer1lA4a2StaofJ7"), storey(4)).
is_a(product("02NiuWFer1lA4a2StaofJ5"), column).
containedIn(product("02NiuWFer1lA4a2StaofJ5"), storey(4)).
is_a(product("02NiuWFer1lA4a2StaofSC"), column).
containedIn(product("02NiuWFer1lA4a2StaofSC"), storey(5)).
is_a(product("02NiuWFer1lA4a2StaofS2"), column).
containedIn(product("02NiuWFer1lA4a2StaofS2"), storey(5)).
is_a(product("02NiuWFer1lA4a2StaofS0"), column).
containedIn(product("02NiuWFer1lA4a2StaofS0"), storey(5)).
is_a(product("02NiuWFer1lA4a2StaofS6"), column).
containedIn(product("02NiuWFer1lA4a2StaofS6"), storey(5)).
is_a(product("02NiuWFer1lA4a2StaofS4"), column).
containedIn(product("02NiuWFer1lA4a2StaofS4"), storey(5)).
is_a(product("02NiuWFer1lA4a2StaofSQ"), column).
containedIn(product("02NiuWFer1lA4a2StaofSQ"), storey(5)).
is_a(product("02NiuWFer1lA4a2StaofSO"), column).
containedIn(product("02NiuWFer1lA4a2StaofSO"), storey(5)).
is_a(product("02NiuWFer1lA4a2StaofSU"), column).
containedIn(product("02NiuWFer1lA4a2StaofSU"), storey(5)).
is_a(product("02NiuWFer1lA4a2StaofSS"), column).
containedIn(product("02NiuWFer1lA4a2StaofSS"), storey(5)).
is_a(product("02NiuWFer1lA4a2StaofSI"), column).
containedIn(product("02NiuWFer1lA4a2StaofSI"), storey(5)).
is_a(product("02NiuWFer1lA4a2StaofSG"), column).
containedIn(product("02NiuWFer1lA4a2StaofSG"), storey(5)).
is_a(product("02NiuWFer1lA4a2StaofSM"), column).
containedIn(product("02NiuWFer1lA4a2StaofSM"), storey(5)).
is_a(product("02NiuWFer1lA4a2StaofSK"), column).
containedIn(product("02NiuWFer1lA4a2StaofSK"), storey(5)).
is_a(product("02NiuWFer1lA4a2StaofTg"), column).
containedIn(product("02NiuWFer1lA4a2StaofTg"), storey(5)).
is_a(product("02NiuWFer1lA4a2StaofTe"), column).
containedIn(product("02NiuWFer1lA4a2StaofTe"), storey(5)).
is_a(product("02NiuWFer1lA4a2StaofTk"), column).
containedIn(product("02NiuWFer1lA4a2StaofTk"), storey(5)).
is_a(product("02NiuWFer1lA4a2StaofTL"), column).
containedIn(product("02NiuWFer1lA4a2StaofTL"), storey(6)).
is_a(product("02NiuWFer1lA4a2StaofUh"), column).
containedIn(product("02NiuWFer1lA4a2StaofUh"), storey(6)).
is_a(product("02NiuWFer1lA4a2StaofUf"), column).
containedIn(product("02NiuWFer1lA4a2StaofUf"), storey(6)).
is_a(product("02NiuWFer1lA4a2StaofUl"), column).
containedIn(product("02NiuWFer1lA4a2StaofUl"), storey(6)).
is_a(product("02NiuWFer1lA4a2StaofUj"), column).
containedIn(product("02NiuWFer1lA4a2StaofUj"), storey(6)).
is_a(product("02NiuWFer1lA4a2StaofUZ"), column).
containedIn(product("02NiuWFer1lA4a2StaofUZ"), storey(6)).
is_a(product("02NiuWFer1lA4a2StaofUX"), column).
containedIn(product("02NiuWFer1lA4a2StaofUX"), storey(6)).
is_a(product("02NiuWFer1lA4a2StaofUd"), column).
containedIn(product("02NiuWFer1lA4a2StaofUd"), storey(6)).
is_a(product("02NiuWFer1lA4a2StaofUb"), column).
containedIn(product("02NiuWFer1lA4a2StaofUb"), storey(6)).
is_a(product("02NiuWFer1lA4a2StaofUx"), column).
containedIn(product("02NiuWFer1lA4a2StaofUx"), storey(6)).
is_a(product("02NiuWFer1lA4a2StaofUv"), column).
containedIn(product("02NiuWFer1lA4a2StaofUv"), storey(6)).
is_a(product("02NiuWFer1lA4a2StaofU$"), column).
containedIn(product("02NiuWFer1lA4a2StaofU$"), storey(6)).
is_a(product("02NiuWFer1lA4a2StaofUz"), column).
containedIn(product("02NiuWFer1lA4a2StaofUz"), storey(6)).
is_a(product("02NiuWFer1lA4a2StaofUp"), column).
containedIn(product("02NiuWFer1lA4a2StaofUp"), storey(6)).
is_a(product("02NiuWFer1lA4a2StaofUn"), column).
containedIn(product("02NiuWFer1lA4a2StaofUn"), storey(6)).
is_a(product("02NiuWFer1lA4a2StaofUt"), column).
containedIn(product("02NiuWFer1lA4a2StaofUt"), storey(6)).
is_a(product("02NiuWFer1lA4a2StaofOP"), column).
containedIn(product("02NiuWFer1lA4a2StaofOP"), storey(1)).
is_a(product("02NiuWFer1lA4a2StaofOV"), column).
containedIn(product("02NiuWFer1lA4a2StaofOV"), storey(1)).
is_a(product("02NiuWFer1lA4a2StaofOT"), column).
containedIn(product("02NiuWFer1lA4a2StaofOT"), storey(1)).
is_a(product("02NiuWFer1lA4a2StaofOJ"), column).
containedIn(product("02NiuWFer1lA4a2StaofOJ"), storey(1)).
is_a(product("02NiuWFer1lA4a2StaofOH"), column).
containedIn(product("02NiuWFer1lA4a2StaofOH"), storey(1)).
is_a(product("02NiuWFer1lA4a2StaofON"), column).
containedIn(product("02NiuWFer1lA4a2StaofON"), storey(1)).
is_a(product("02NiuWFer1lA4a2StaofOL"), column).
containedIn(product("02NiuWFer1lA4a2StaofOL"), storey(1)).
is_a(product("02NiuWFer1lA4a2StaofPh"), column).
containedIn(product("02NiuWFer1lA4a2StaofPh"), storey(1)).
is_a(product("02NiuWFer1lA4a2StaofPf"), column).
containedIn(product("02NiuWFer1lA4a2StaofPf"), storey(1)).
is_a(product("02NiuWFer1lA4a2StaofPl"), column).
containedIn(product("02NiuWFer1lA4a2StaofPl"), storey(1)).
is_a(product("02NiuWFer1lA4a2StaofPj"), column).
containedIn(product("02NiuWFer1lA4a2StaofPj"), storey(1)).
is_a(product("02NiuWFer1lA4a2StaofPZ"), column).
containedIn(product("02NiuWFer1lA4a2StaofPZ"), storey(1)).
is_a(product("02NiuWFer1lA4a2StaofPX"), column).
containedIn(product("02NiuWFer1lA4a2StaofPX"), storey(1)).
is_a(product("02NiuWFer1lA4a2StaofPd"), column).
containedIn(product("02NiuWFer1lA4a2StaofPd"), storey(1)).
is_a(product("02NiuWFer1lA4a2StaofPb"), column).
containedIn(product("02NiuWFer1lA4a2StaofPb"), storey(1)).
is_a(product("02NiuWFer1lA4a2StaofPx"), column).
containedIn(product("02NiuWFer1lA4a2StaofPx"), storey(1)).
is_a(product("02NiuWFer1lA4a2Staofbl"), slab).
containedIn(product("02NiuWFer1lA4a2Staofbl"), storey(3)).
is_a(product("02NiuWFer1lA4a2Staofbc"), slab).
containedIn(product("02NiuWFer1lA4a2Staofbc"), storey(3)).
is_a(product("02NiuWFer1lA4a2Staofbz"), slab).
containedIn(product("02NiuWFer1lA4a2Staofbz"), storey(3)).
is_a(product("02NiuWFer1lA4a2Staofbq"), slab).
containedIn(product("02NiuWFer1lA4a2Staofbq"), storey(3)).
is_a(product("02NiuWFer1lA4a2Staofc0"), slab).
containedIn(product("02NiuWFer1lA4a2Staofc0"), storey(4)).
is_a(product("02NiuWFer1lA4a2StaofcV"), slab).
containedIn(product("02NiuWFer1lA4a2StaofcV"), storey(4)).
is_a(product("02NiuWFer1lA4a2StaofcM"), slab).
containedIn(product("02NiuWFer1lA4a2StaofcM"), storey(4)).
is_a(product("02NiuWFer1lA4a2Staofdj"), slab).
containedIn(product("02NiuWFer1lA4a2Staofdj"), storey(4)).
is_a(product("02NiuWFer1lA4a2StaofWv"), slab).
containedIn(product("02NiuWFer1lA4a2StaofWv"), storey(5)).
is_a(product("02NiuWFer1lA4a2StaofWm"), slab).
containedIn(product("02NiuWFer1lA4a2StaofWm"), storey(5)).
is_a(product("02NiuWFer1lA4a2StaofWF"), slab).
containedIn(product("02NiuWFer1lA4a2StaofWF"), storey(5)).
is_a(product("02NiuWFer1lA4a2StaofW6"), slab).
containedIn(product("02NiuWFer1lA4a2StaofW6"), storey(5)).
is_a(product("02NiuWFer1lA4a2StaofXI"), slab).
containedIn(product("02NiuWFer1lA4a2StaofXI"), storey(6)).
is_a(product("02NiuWFer1lA4a2StaofYf"), slab).
containedIn(product("02NiuWFer1lA4a2StaofYf"), storey(6)).
is_a(product("02NiuWFer1lA4a2StaofYW"), slab).
containedIn(product("02NiuWFer1lA4a2StaofYW"), storey(6)).
is_a(product("02NiuWFer1lA4a2StaofY$"), slab).
containedIn(product("02NiuWFer1lA4a2StaofY$"), storey(6)).
is_a(product("02NiuWFer1lA4a2StaofZp"), slab).
containedIn(product("02NiuWFer1lA4a2StaofZp"), storey(7)).
is_a(product("02NiuWFer1lA4a2StaofZA"), slab).
containedIn(product("02NiuWFer1lA4a2StaofZA"), storey(7)).
is_a(product("02NiuWFer1lA4a2StaofZ1"), slab).
containedIn(product("02NiuWFer1lA4a2StaofZ1"), storey(7)).
is_a(product("02NiuWFer1lA4a2StaofZO"), slab).
containedIn(product("02NiuWFer1lA4a2StaofZO"), storey(7)).
is_a(product("0RvmDCxjLDTADBjEF5c1$4"), wall).
containedIn(product("0RvmDCxjLDTADBjEF5c1$4"), storey(0)).
is_a(product("0RvmDCxjLDTADBjEF5c1$5"), wall).
containedIn(product("0RvmDCxjLDTADBjEF5c1$5"), storey(0)).
is_a(product("0RvmDCxjLDTADBjEF5c1$A"), wall).
containedIn(product("0RvmDCxjLDTADBjEF5c1$A"), storey(0)).
is_a(product("0RvmDCxjLDTADBjEF5c1$B"), wall).
containedIn(product("0RvmDCxjLDTADBjEF5c1$B"), storey(0)).
is_a(product("0RvmDCxjLDTADBjEF5c1_C"), slab).
containedIn(product("0RvmDCxjLDTADBjEF5c1_C"), storey(0)).
is_a(product("0N06MRuuX5WQjOvyYDy0T6"), wall).
containedIn(product("0N06MRuuX5WQjOvyYDy0T6"), storey(7)).
is_a(product("0N06MRuuX5WQjOvyYDy0T9"), wall).
containedIn(product("0N06MRuuX5WQjOvyYDy0T9"), storey(7)).
is_a(product("0N06MRuuX5WQjOvyYDy0T8"), wall).
containedIn(product("0N06MRuuX5WQjOvyYDy0T8"), storey(7)).
is_a(product("0N06MRuuX5WQjOvyYDy0TB"), wall).
containedIn(product("0N06MRuuX5WQjOvyYDy0TB"), storey(7)).
is_a(product("0N06MRuuX5WQjOvyYDy0TF"), column).
containedIn(product("0N06MRuuX5WQjOvyYDy0TF"), storey(7)).
is_a(product("0N06MRuuX5WQjOvyYDy0Qn"), column).
containedIn(product("0N06MRuuX5WQjOvyYDy0Qn"), storey(7)).
is_a(product("0N06MRuuX5WQjOvyYDy0Qp"), column).
containedIn(product("0N06MRuuX5WQjOvyYDy0Qp"), storey(7)).
is_a(product("0N06MRuuX5WQjOvyYDy0Qr"), column).
containedIn(product("0N06MRuuX5WQjOvyYDy0Qr"), storey(7)).
is_a(product("0N06MRuuX5WQjOvyYDy0Qt"), column).
containedIn(product("0N06MRuuX5WQjOvyYDy0Qt"), storey(7)).
is_a(product("0N06MRuuX5WQjOvyYDy0Qv"), column).
containedIn(product("0N06MRuuX5WQjOvyYDy0Qv"), storey(7)).
is_a(product("0N06MRuuX5WQjOvyYDy0Qx"), column).
containedIn(product("0N06MRuuX5WQjOvyYDy0Qx"), storey(7)).
is_a(product("0N06MRuuX5WQjOvyYDy0Qz"), column).
containedIn(product("0N06MRuuX5WQjOvyYDy0Qz"), storey(7)).
is_a(product("0N06MRuuX5WQjOvyYDy0Q$"), column).
containedIn(product("0N06MRuuX5WQjOvyYDy0Q$"), storey(7)).
is_a(product("0N06MRuuX5WQjOvyYDy0QX"), column).
containedIn(product("0N06MRuuX5WQjOvyYDy0QX"), storey(7)).
is_a(product("0N06MRuuX5WQjOvyYDy0QZ"), column).
containedIn(product("0N06MRuuX5WQjOvyYDy0QZ"), storey(7)).
is_a(product("0N06MRuuX5WQjOvyYDy0Qb"), column).
containedIn(product("0N06MRuuX5WQjOvyYDy0Qb"), storey(7)).
is_a(product("0N06MRuuX5WQjOvyYDy0Qd"), column).
containedIn(product("0N06MRuuX5WQjOvyYDy0Qd"), storey(7)).
is_a(product("0N06MRuuX5WQjOvyYDy0Qf"), column).
containedIn(product("0N06MRuuX5WQjOvyYDy0Qf"), storey(7)).
is_a(product("0N06MRuuX5WQjOvyYDy0Qh"), column).
containedIn(product("0N06MRuuX5WQjOvyYDy0Qh"), storey(7)).
is_a(product("0N06MRuuX5WQjOvyYDy0Qj"), column).
containedIn(product("0N06MRuuX5WQjOvyYDy0Qj"), storey(7)).
is_a(product("0N06MRuuX5WQjOvyYDy0QH"), slab).
containedIn(product("0N06MRuuX5WQjOvyYDy0QH"), storey(8)).
is_a(product("0N06MRuuX5WQjOvyYDy0QO"), slab).
containedIn(product("0N06MRuuX5WQjOvyYDy0QO"), storey(8)).
is_a(product("0N06MRuuX5WQjOvyYDy0Q3"), slab).
containedIn(product("0N06MRuuX5WQjOvyYDy0Q3"), storey(8)).
is_a(product("0N06MRuuX5WQjOvyYDy0QA"), slab).
containedIn(product("0N06MRuuX5WQjOvyYDy0QA"), storey(8)).
is_a(product("0N06MRuuX5WQjOvyYDy0Oz"), wall).
containedIn(product("0N06MRuuX5WQjOvyYDy0Oz"), storey(8)).
is_a(product("0N06MRuuX5WQjOvyYDy0Oy"), wall).
containedIn(product("0N06MRuuX5WQjOvyYDy0Oy"), storey(8)).
is_a(product("0N06MRuuX5WQjOvyYDy0O$"), wall).
containedIn(product("0N06MRuuX5WQjOvyYDy0O$"), storey(8)).
is_a(product("0N06MRuuX5WQjOvyYDy0O_"), wall).
containedIn(product("0N06MRuuX5WQjOvyYDy0O_"), storey(8)).
is_a(product("0N06MRuuX5WQjOvyYDy0OY"), column).
containedIn(product("0N06MRuuX5WQjOvyYDy0OY"), storey(8)).
is_a(product("0N06MRuuX5WQjOvyYDy0Oa"), column).
containedIn(product("0N06MRuuX5WQjOvyYDy0Oa"), storey(8)).
is_a(product("0N06MRuuX5WQjOvyYDy0Oc"), column).
containedIn(product("0N06MRuuX5WQjOvyYDy0Oc"), storey(8)).
is_a(product("0N06MRuuX5WQjOvyYDy0Oe"), column).
containedIn(product("0N06MRuuX5WQjOvyYDy0Oe"), storey(8)).
is_a(product("0N06MRuuX5WQjOvyYDy0Og"), column).
containedIn(product("0N06MRuuX5WQjOvyYDy0Og"), storey(8)).
is_a(product("0N06MRuuX5WQjOvyYDy0Oi"), column).
containedIn(product("0N06MRuuX5WQjOvyYDy0Oi"), storey(8)).
is_a(product("0N06MRuuX5WQjOvyYDy0Ok"), column).
containedIn(product("0N06MRuuX5WQjOvyYDy0Ok"), storey(8)).
is_a(product("0N06MRuuX5WQjOvyYDy0OG"), column).
containedIn(product("0N06MRuuX5WQjOvyYDy0OG"), storey(8)).
is_a(product("0N06MRuuX5WQjOvyYDy0OI"), column).
containedIn(product("0N06MRuuX5WQjOvyYDy0OI"), storey(8)).
is_a(product("0N06MRuuX5WQjOvyYDy0OK"), column).
containedIn(product("0N06MRuuX5WQjOvyYDy0OK"), storey(8)).
is_a(product("0N06MRuuX5WQjOvyYDy0OM"), column).
containedIn(product("0N06MRuuX5WQjOvyYDy0OM"), storey(8)).
is_a(product("0N06MRuuX5WQjOvyYDy0OO"), column).
containedIn(product("0N06MRuuX5WQjOvyYDy0OO"), storey(8)).
is_a(product("0N06MRuuX5WQjOvyYDy0OQ"), column).
containedIn(product("0N06MRuuX5WQjOvyYDy0OQ"), storey(8)).
is_a(product("0N06MRuuX5WQjOvyYDy0OS"), column).
containedIn(product("0N06MRuuX5WQjOvyYDy0OS"), storey(8)).
is_a(product("0N06MRuuX5WQjOvyYDy0OU"), column).
containedIn(product("0N06MRuuX5WQjOvyYDy0OU"), storey(8)).
is_a(product("0N06MRuuX5WQjOvyYDy0O0"), column).
containedIn(product("0N06MRuuX5WQjOvyYDy0O0"), storey(8)).
is_a(product("0N06MRuuX5WQjOvyYDy0O4"), slab).
containedIn(product("0N06MRuuX5WQjOvyYDy0O4"), storey(9)).
is_a(product("0N06MRuuX5WQjOvyYDy0OF"), slab).
containedIn(product("0N06MRuuX5WQjOvyYDy0OF"), storey(9)).
is_a(product("0N06MRuuX5WQjOvyYDy0Ps"), slab).
containedIn(product("0N06MRuuX5WQjOvyYDy0Ps"), storey(9)).
is_a(product("0N06MRuuX5WQjOvyYDy0PX"), slab).
containedIn(product("0N06MRuuX5WQjOvyYDy0PX"), storey(9)).
is_a(product("0N06MRuuX5WQjOvyYDy0MJ"), wall).
containedIn(product("0N06MRuuX5WQjOvyYDy0MJ"), storey(9)).
is_a(product("0N06MRuuX5WQjOvyYDy0MI"), wall).
containedIn(product("0N06MRuuX5WQjOvyYDy0MI"), storey(9)).
is_a(product("0N06MRuuX5WQjOvyYDy0ML"), wall).
containedIn(product("0N06MRuuX5WQjOvyYDy0ML"), storey(9)).
is_a(product("0N06MRuuX5WQjOvyYDy0MK"), wall).
containedIn(product("0N06MRuuX5WQjOvyYDy0MK"), storey(9)).
is_a(product("0N06MRuuX5WQjOvyYDy0MO"), column).
containedIn(product("0N06MRuuX5WQjOvyYDy0MO"), storey(9)).
is_a(product("0N06MRuuX5WQjOvyYDy0MQ"), column).
containedIn(product("0N06MRuuX5WQjOvyYDy0MQ"), storey(9)).
is_a(product("0N06MRuuX5WQjOvyYDy0MS"), column).
containedIn(product("0N06MRuuX5WQjOvyYDy0MS"), storey(9)).
is_a(product("0N06MRuuX5WQjOvyYDy0MU"), column).
containedIn(product("0N06MRuuX5WQjOvyYDy0MU"), storey(9)).
is_a(product("0N06MRuuX5WQjOvyYDy0M0"), column).
containedIn(product("0N06MRuuX5WQjOvyYDy0M0"), storey(9)).
is_a(product("0N06MRuuX5WQjOvyYDy0M2"), column).
containedIn(product("0N06MRuuX5WQjOvyYDy0M2"), storey(9)).
is_a(product("0N06MRuuX5WQjOvyYDy0M4"), column).
containedIn(product("0N06MRuuX5WQjOvyYDy0M4"), storey(9)).
is_a(product("0N06MRuuX5WQjOvyYDy0M6"), column).
containedIn(product("0N06MRuuX5WQjOvyYDy0M6"), storey(9)).
is_a(product("0N06MRuuX5WQjOvyYDy0M8"), column).
containedIn(product("0N06MRuuX5WQjOvyYDy0M8"), storey(9)).
is_a(product("0N06MRuuX5WQjOvyYDy0MA"), column).
containedIn(product("0N06MRuuX5WQjOvyYDy0MA"), storey(9)).
is_a(product("0N06MRuuX5WQjOvyYDy0MC"), column).
containedIn(product("0N06MRuuX5WQjOvyYDy0MC"), storey(9)).
is_a(product("0N06MRuuX5WQjOvyYDy0ME"), column).
containedIn(product("0N06MRuuX5WQjOvyYDy0ME"), storey(9)).
is_a(product("0N06MRuuX5WQjOvyYDy0Nm"), column).
containedIn(product("0N06MRuuX5WQjOvyYDy0Nm"), storey(9)).
is_a(product("0N06MRuuX5WQjOvyYDy0No"), column).
containedIn(product("0N06MRuuX5WQjOvyYDy0No"), storey(9)).
is_a(product("0N06MRuuX5WQjOvyYDy0Nq"), column).
containedIn(product("0N06MRuuX5WQjOvyYDy0Nq"), storey(9)).
is_a(product("0N06MRuuX5WQjOvyYDy0Ns"), column).
containedIn(product("0N06MRuuX5WQjOvyYDy0Ns"), storey(9)).
is_a(product("0N06MRuuX5WQjOvyYDy0Nw"), slab).
containedIn(product("0N06MRuuX5WQjOvyYDy0Nw"), storey(10)).
is_a(product("0N06MRuuX5WQjOvyYDy0Nb"), slab).
containedIn(product("0N06MRuuX5WQjOvyYDy0Nb"), storey(10)).
is_a(product("0N06MRuuX5WQjOvyYDy0Ni"), slab).
containedIn(product("0N06MRuuX5WQjOvyYDy0Ni"), storey(10)).
is_a(product("0N06MRuuX5WQjOvyYDy0NN"), slab).
containedIn(product("0N06MRuuX5WQjOvyYDy0NN"), storey(10)).
is_a(product("0N06MRuuX5WQjOvyYDy3lQ"), wall).
containedIn(product("0N06MRuuX5WQjOvyYDy3lQ"), storey(10)).
is_a(product("0N06MRuuX5WQjOvyYDy3lT"), wall).
containedIn(product("0N06MRuuX5WQjOvyYDy3lT"), storey(10)).
is_a(product("0N06MRuuX5WQjOvyYDy3lS"), wall).
containedIn(product("0N06MRuuX5WQjOvyYDy3lS"), storey(10)).
is_a(product("0N06MRuuX5WQjOvyYDy3lV"), wall).
containedIn(product("0N06MRuuX5WQjOvyYDy3lV"), storey(10)).
is_a(product("0N06MRuuX5WQjOvyYDy3l3"), column).
containedIn(product("0N06MRuuX5WQjOvyYDy3l3"), storey(10)).
is_a(product("0N06MRuuX5WQjOvyYDy3l5"), column).
containedIn(product("0N06MRuuX5WQjOvyYDy3l5"), storey(10)).
is_a(product("0N06MRuuX5WQjOvyYDy3l7"), column).
containedIn(product("0N06MRuuX5WQjOvyYDy3l7"), storey(10)).
is_a(product("0N06MRuuX5WQjOvyYDy3l9"), column).
containedIn(product("0N06MRuuX5WQjOvyYDy3l9"), storey(10)).
is_a(product("0N06MRuuX5WQjOvyYDy3lB"), column).
containedIn(product("0N06MRuuX5WQjOvyYDy3lB"), storey(10)).
is_a(product("0N06MRuuX5WQjOvyYDy3lD"), column).
containedIn(product("0N06MRuuX5WQjOvyYDy3lD"), storey(10)).
is_a(product("0N06MRuuX5WQjOvyYDy3lF"), column).
containedIn(product("0N06MRuuX5WQjOvyYDy3lF"), storey(10)).
is_a(product("0N06MRuuX5WQjOvyYDy3in"), column).
containedIn(product("0N06MRuuX5WQjOvyYDy3in"), storey(10)).
is_a(product("0N06MRuuX5WQjOvyYDy3ip"), column).
containedIn(product("0N06MRuuX5WQjOvyYDy3ip"), storey(10)).
is_a(product("0N06MRuuX5WQjOvyYDy3ir"), column).
containedIn(product("0N06MRuuX5WQjOvyYDy3ir"), storey(10)).
is_a(product("0N06MRuuX5WQjOvyYDy3it"), column).
containedIn(product("0N06MRuuX5WQjOvyYDy3it"), storey(10)).
is_a(product("0N06MRuuX5WQjOvyYDy3iv"), column).
containedIn(product("0N06MRuuX5WQjOvyYDy3iv"), storey(10)).
is_a(product("0N06MRuuX5WQjOvyYDy3ix"), column).
containedIn(product("0N06MRuuX5WQjOvyYDy3ix"), storey(10)).
is_a(product("0N06MRuuX5WQjOvyYDy3iz"), column).
containedIn(product("0N06MRuuX5WQjOvyYDy3iz"), storey(10)).
is_a(product("0N06MRuuX5WQjOvyYDy3i$"), column).
containedIn(product("0N06MRuuX5WQjOvyYDy3i$"), storey(10)).
is_a(product("0N06MRuuX5WQjOvyYDy3iX"), column).
containedIn(product("0N06MRuuX5WQjOvyYDy3iX"), storey(10)).
is_a(product("0N06MRuuX5WQjOvyYDy3ib"), slab).
containedIn(product("0N06MRuuX5WQjOvyYDy3ib"), storey(11)).
is_a(product("0N06MRuuX5WQjOvyYDy3ii"), slab).
containedIn(product("0N06MRuuX5WQjOvyYDy3ii"), storey(11)).
is_a(product("0N06MRuuX5WQjOvyYDy3iN"), slab).
containedIn(product("0N06MRuuX5WQjOvyYDy3iN"), storey(11)).
is_a(product("0N06MRuuX5WQjOvyYDy3iU"), slab).
containedIn(product("0N06MRuuX5WQjOvyYDy3iU"), storey(11)).
is_a(product("0N06MRuuX5WQjOvyYDy3iE"), slab).
containedIn(product("0N06MRuuX5WQjOvyYDy3iE"), storey(12)).
is_a(product("0A8QdCd812$xV$VdhPItzP"), slab).
containedIn(product("0A8QdCd812$xV$VdhPItzP"), storey(site)).
is_a(product("0A8QdCd812$xV$VdhPItzI"), slab).
containedIn(product("0A8QdCd812$xV$VdhPItzI"), storey(site)).




belongsTo(product("3eHkYig055Ex7oFuhZVTBc"), task("SL02D")).
belongsTo(product("3eHkYig055Ex7oFuhZVTAV"), task("SL02C")).
belongsTo(product("3eHkYig055Ex7oFuhZVTAb"), task("SL02A")).
belongsTo(product("3eHkYig055Ex7oFuhZVT98"), task("SL02B")).
belongsTo(product("3eHkYig055Ex7oFuhZVIqN"), task("EL01")).
belongsTo(product("3eHkYig055Ex7oFuhZVIqM"), task("EL01")).
belongsTo(product("3eHkYig055Ex7oFuhZVIqf"), task("EL01")).
belongsTo(product("3eHkYig055Ex7oFuhZVIqe"), task("EL01")).
belongsTo(product("3eHkYig055Ex7oFuhZVInm"), task("EL02")).
belongsTo(product("3eHkYig055Ex7oFuhZVInp"), task("EL02")).
belongsTo(product("3eHkYig055Ex7oFuhZVIno"), task("EL02")).
belongsTo(product("3eHkYig055Ex7oFuhZVInr"), task("EL02")).
belongsTo(product("3eHkYig055Ex7oFuhZVI_Q"), task("EL03")).
belongsTo(product("3eHkYig055Ex7oFuhZVI_T"), task("EL03")).
belongsTo(product("3eHkYig055Ex7oFuhZVI_S"), task("EL03")).
belongsTo(product("3eHkYig055Ex7oFuhZVI_V"), task("EL03")).
belongsTo(product("3eHkYig055Ex7oFuhZVIya"), task("EL04")).
belongsTo(product("3eHkYig055Ex7oFuhZVIyd"), task("EL04")).
belongsTo(product("3eHkYig055Ex7oFuhZVIyc"), task("EL04")).
belongsTo(product("3eHkYig055Ex7oFuhZVIyv"), task("EL04")).
belongsTo(product("3eHkYig055Ex7oFuhZVIvF"), task("EL05")).
belongsTo(product("3eHkYig055Ex7oFuhZVIvE"), task("EL05")).
belongsTo(product("3eHkYig055Ex7oFuhZVIv1"), task("EL05")).
belongsTo(product("3eHkYig055Ex7oFuhZVIv0"), task("EL05")).
belongsTo(product("3eHkYig055Ex7oFuhZVId2"), task("EL06")).
belongsTo(product("3eHkYig055Ex7oFuhZVId5"), task("EL06")).
belongsTo(product("3eHkYig055Ex7oFuhZVId4"), task("EL06")).
belongsTo(product("3eHkYig055Ex7oFuhZVId7"), task("EL06")).
belongsTo(product("3eHkYig055Ex7oFuhZVIZL"), task("SL01A")).
belongsTo(product("3eHkYig055Ex7oFuhZVIYt"), task("SL01B")).
belongsTo(product("3eHkYig055Ex7oFuhZVIW2"), task("SL01C")).
belongsTo(product("2bxL$O5Cz3xOWd38LvRNOO"), task("SL01D")).
belongsTo(product("2AICztFX95Z8QtA12iaPQo"), task("CL02D")).
belongsTo(product("2AICztFX95Z8QtA12iaPQq"), task("CL02C")).
belongsTo(product("2AICztFX95Z8QtA12iaPQs"), task("CL02B")).
belongsTo(product("2AICztFX95Z8QtA12iaPR8"), task("CL02A")).
belongsTo(product("2AICztFX95Z8QtA12iaPRA"), task("CL02A")).
belongsTo(product("2AICztFX95Z8QtA12iaPRC"), task("CL02C")).
belongsTo(product("2AICztFX95Z8QtA12iaPRE"), task("CL02D")).
belongsTo(product("2AICztFX95Z8QtA12iaPR0"), task("CL02C")).
belongsTo(product("2AICztFX95Z8QtA12iaPR2"), task("CL02A")).
belongsTo(product("2AICztFX95Z8QtA12iaPR4"), task("CL02B")).
belongsTo(product("2AICztFX95Z8QtA12iaPR6"), task("CL02B")).
belongsTo(product("2AICztFX95Z8QtA12iaPRO"), task("CL02D")).
belongsTo(product("3eORFXfAH04e_QSwbI$PvJ"), task("CL02A")).
belongsTo(product("3eORFXfAH04e_QSwbI$P_Y"), task("CL02B")).
belongsTo(product("3eORFXfAH04e_QSwbI$P$5"), task("CL02D")).
belongsTo(product("3eORFXfAH04e_QSwbI$P$3"), task("CL02C")).
belongsTo(product("02NiuWFer1lA4a2StaofHI"), task("CL03D")).
belongsTo(product("02NiuWFer1lA4a2StaofHG"), task("CL03C")).
belongsTo(product("02NiuWFer1lA4a2StaofHM"), task("CL03B")).
belongsTo(product("02NiuWFer1lA4a2StaofHK"), task("CL03A")).
belongsTo(product("02NiuWFer1lA4a2StaofIg"), task("CL03A")).
belongsTo(product("02NiuWFer1lA4a2StaofIe"), task("CL03C")).
belongsTo(product("02NiuWFer1lA4a2StaofIk"), task("CL03D")).
belongsTo(product("02NiuWFer1lA4a2StaofIi"), task("CL03C")).
belongsTo(product("02NiuWFer1lA4a2StaofIY"), task("CL03A")).
belongsTo(product("02NiuWFer1lA4a2StaofIW"), task("CL03B")).
belongsTo(product("02NiuWFer1lA4a2StaofIc"), task("CL03B")).
belongsTo(product("02NiuWFer1lA4a2StaofIa"), task("CL03D")).
belongsTo(product("02NiuWFer1lA4a2StaofIw"), task("CL03A")).
belongsTo(product("02NiuWFer1lA4a2StaofIu"), task("CL03B")).
belongsTo(product("02NiuWFer1lA4a2StaofI_"), task("CL03D")).
belongsTo(product("02NiuWFer1lA4a2StaofIy"), task("CL03C")).
belongsTo(product("02NiuWFer1lA4a2StaofJx"), task("CL04D")).
belongsTo(product("02NiuWFer1lA4a2StaofJv"), task("CL04C")).
belongsTo(product("02NiuWFer1lA4a2StaofJ$"), task("CL04B")).
belongsTo(product("02NiuWFer1lA4a2StaofJz"), task("CL04A")).
belongsTo(product("02NiuWFer1lA4a2StaofJp"), task("CL04A")).
belongsTo(product("02NiuWFer1lA4a2StaofJn"), task("CL04C")).
belongsTo(product("02NiuWFer1lA4a2StaofJt"), task("CL04D")).
belongsTo(product("02NiuWFer1lA4a2StaofJr"), task("CL04C")).
belongsTo(product("02NiuWFer1lA4a2StaofJB"), task("CL04A")).
belongsTo(product("02NiuWFer1lA4a2StaofJ9"), task("CL04B")).
belongsTo(product("02NiuWFer1lA4a2StaofJF"), task("CL04B")).
belongsTo(product("02NiuWFer1lA4a2StaofJD"), task("CL04D")).
belongsTo(product("02NiuWFer1lA4a2StaofJ3"), task("CL04A")).
belongsTo(product("02NiuWFer1lA4a2StaofJ1"), task("CL04B")).
belongsTo(product("02NiuWFer1lA4a2StaofJ7"), task("CL04D")).
belongsTo(product("02NiuWFer1lA4a2StaofJ5"), task("CL04C")).
belongsTo(product("02NiuWFer1lA4a2StaofSC"), task("CL05D")).
belongsTo(product("02NiuWFer1lA4a2StaofS2"), task("CL05C")).
belongsTo(product("02NiuWFer1lA4a2StaofS0"), task("CL05B")).
belongsTo(product("02NiuWFer1lA4a2StaofS6"), task("CL05A")).
belongsTo(product("02NiuWFer1lA4a2StaofS4"), task("CL05A")).
belongsTo(product("02NiuWFer1lA4a2StaofSQ"), task("CL05C")).
belongsTo(product("02NiuWFer1lA4a2StaofSO"), task("CL05D")).
belongsTo(product("02NiuWFer1lA4a2StaofSU"), task("CL05C")).
belongsTo(product("02NiuWFer1lA4a2StaofSS"), task("CL05A")).
belongsTo(product("02NiuWFer1lA4a2StaofSI"), task("CL05B")).
belongsTo(product("02NiuWFer1lA4a2StaofSG"), task("CL05B")).
belongsTo(product("02NiuWFer1lA4a2StaofSM"), task("CL05D")).
belongsTo(product("02NiuWFer1lA4a2StaofSK"), task("CL05A")).
belongsTo(product("02NiuWFer1lA4a2StaofTg"), task("CL05B")).
belongsTo(product("02NiuWFer1lA4a2StaofTe"), task("CL05D")).
belongsTo(product("02NiuWFer1lA4a2StaofTk"), task("CL05C")).
belongsTo(product("02NiuWFer1lA4a2StaofTL"), task("CL06D")).
belongsTo(product("02NiuWFer1lA4a2StaofUh"), task("CL06C")).
belongsTo(product("02NiuWFer1lA4a2StaofUf"), task("CL06B")).
belongsTo(product("02NiuWFer1lA4a2StaofUl"), task("CL06A")).
belongsTo(product("02NiuWFer1lA4a2StaofUj"), task("CL06A")).
belongsTo(product("02NiuWFer1lA4a2StaofUZ"), task("CL06C")).
belongsTo(product("02NiuWFer1lA4a2StaofUX"), task("CL06D")).
belongsTo(product("02NiuWFer1lA4a2StaofUd"), task("CL06C")).
belongsTo(product("02NiuWFer1lA4a2StaofUb"), task("CL06A")).
belongsTo(product("02NiuWFer1lA4a2StaofUx"), task("CL06B")).
belongsTo(product("02NiuWFer1lA4a2StaofUv"), task("CL06B")).
belongsTo(product("02NiuWFer1lA4a2StaofU$"), task("CL06D")).
belongsTo(product("02NiuWFer1lA4a2StaofUz"), task("CL06A")).
belongsTo(product("02NiuWFer1lA4a2StaofUp"), task("CL06B")).
belongsTo(product("02NiuWFer1lA4a2StaofUn"), task("CL06D")).
belongsTo(product("02NiuWFer1lA4a2StaofUt"), task("CL06C")).
belongsTo(product("02NiuWFer1lA4a2StaofOP"), task("CL01D")).
belongsTo(product("02NiuWFer1lA4a2StaofOV"), task("CL01C")).
belongsTo(product("02NiuWFer1lA4a2StaofOT"), task("CL01B")).
belongsTo(product("02NiuWFer1lA4a2StaofOJ"), task("CL01A")).
belongsTo(product("02NiuWFer1lA4a2StaofOH"), task("CL01A")).
belongsTo(product("02NiuWFer1lA4a2StaofON"), task("CL01C")).
belongsTo(product("02NiuWFer1lA4a2StaofOL"), task("CL01D")).
belongsTo(product("02NiuWFer1lA4a2StaofPh"), task("CL01C")).
belongsTo(product("02NiuWFer1lA4a2StaofPf"), task("CL01A")).
belongsTo(product("02NiuWFer1lA4a2StaofPl"), task("CL01B")).
belongsTo(product("02NiuWFer1lA4a2StaofPj"), task("CL01B")).
belongsTo(product("02NiuWFer1lA4a2StaofPZ"), task("CL01D")).
belongsTo(product("02NiuWFer1lA4a2StaofPX"), task("CL01A")).
belongsTo(product("02NiuWFer1lA4a2StaofPd"), task("CL01B")).
belongsTo(product("02NiuWFer1lA4a2StaofPb"), task("CL01D")).
belongsTo(product("02NiuWFer1lA4a2StaofPx"), task("CL01C")).
belongsTo(product("02NiuWFer1lA4a2Staofbl"), task("SL03D")).
belongsTo(product("02NiuWFer1lA4a2Staofbc"), task("SL03C")).
belongsTo(product("02NiuWFer1lA4a2Staofbz"), task("SL03A")).
belongsTo(product("02NiuWFer1lA4a2Staofbq"), task("SL03B")).
belongsTo(product("02NiuWFer1lA4a2Staofc0"), task("SL04D")).
belongsTo(product("02NiuWFer1lA4a2StaofcV"), task("SL04C")).
belongsTo(product("02NiuWFer1lA4a2StaofcM"), task("SL04A")).
belongsTo(product("02NiuWFer1lA4a2Staofdj"), task("SL04B")).
belongsTo(product("02NiuWFer1lA4a2StaofWv"), task("SL05D")).
belongsTo(product("02NiuWFer1lA4a2StaofWm"), task("SL05C")).
belongsTo(product("02NiuWFer1lA4a2StaofWF"), task("SL05A")).
belongsTo(product("02NiuWFer1lA4a2StaofW6"), task("SL05B")).
belongsTo(product("02NiuWFer1lA4a2StaofXI"), task("SL06D")).
belongsTo(product("02NiuWFer1lA4a2StaofYf"), task("SL06C")).
belongsTo(product("02NiuWFer1lA4a2StaofYW"), task("SL06A")).
belongsTo(product("02NiuWFer1lA4a2StaofY$"), task("SL06B")).
belongsTo(product("02NiuWFer1lA4a2StaofZp"), task("SL07D")).
belongsTo(product("02NiuWFer1lA4a2StaofZA"), task("SL07C")).
belongsTo(product("02NiuWFer1lA4a2StaofZ1"), task("SL07A")).
belongsTo(product("02NiuWFer1lA4a2StaofZO"), task("SL07B")).
belongsTo(product("0RvmDCxjLDTADBjEF5c1$4"), task("EL00")).
belongsTo(product("0RvmDCxjLDTADBjEF5c1$5"), task("EL00")).
belongsTo(product("0RvmDCxjLDTADBjEF5c1$A"), task("EL00")).
belongsTo(product("0RvmDCxjLDTADBjEF5c1$B"), task("EL00")).
belongsTo(product("0RvmDCxjLDTADBjEF5c1_C"), task("SL00")).
belongsTo(product("0N06MRuuX5WQjOvyYDy0T6"), task("EL07")).
belongsTo(product("0N06MRuuX5WQjOvyYDy0T9"), task("EL07")).
belongsTo(product("0N06MRuuX5WQjOvyYDy0T8"), task("EL07")).
belongsTo(product("0N06MRuuX5WQjOvyYDy0TB"), task("EL07")).
belongsTo(product("0N06MRuuX5WQjOvyYDy0TF"), task("CL07D")).
belongsTo(product("0N06MRuuX5WQjOvyYDy0Qn"), task("CL07C")).
belongsTo(product("0N06MRuuX5WQjOvyYDy0Qp"), task("CL07B")).
belongsTo(product("0N06MRuuX5WQjOvyYDy0Qr"), task("CL07A")).
belongsTo(product("0N06MRuuX5WQjOvyYDy0Qt"), task("CL07A")).
belongsTo(product("0N06MRuuX5WQjOvyYDy0Qv"), task("CL07C")).
belongsTo(product("0N06MRuuX5WQjOvyYDy0Qx"), task("CL07D")).
belongsTo(product("0N06MRuuX5WQjOvyYDy0Qz"), task("CL07C")).
belongsTo(product("0N06MRuuX5WQjOvyYDy0Q$"), task("CL07A")).
belongsTo(product("0N06MRuuX5WQjOvyYDy0QX"), task("CL07B")).
belongsTo(product("0N06MRuuX5WQjOvyYDy0QZ"), task("CL07B")).
belongsTo(product("0N06MRuuX5WQjOvyYDy0Qb"), task("CL07D")).
belongsTo(product("0N06MRuuX5WQjOvyYDy0Qd"), task("CL07A")).
belongsTo(product("0N06MRuuX5WQjOvyYDy0Qf"), task("CL07B")).
belongsTo(product("0N06MRuuX5WQjOvyYDy0Qh"), task("CL07D")).
belongsTo(product("0N06MRuuX5WQjOvyYDy0Qj"), task("CL07C")).
belongsTo(product("0N06MRuuX5WQjOvyYDy0QH"), task("SL08D")).
belongsTo(product("0N06MRuuX5WQjOvyYDy0QO"), task("SL08C")).
belongsTo(product("0N06MRuuX5WQjOvyYDy0Q3"), task("SL08A")).
belongsTo(product("0N06MRuuX5WQjOvyYDy0QA"), task("SL08B")).
belongsTo(product("0N06MRuuX5WQjOvyYDy0Oz"), task("EL08")).
belongsTo(product("0N06MRuuX5WQjOvyYDy0Oy"), task("EL08")).
belongsTo(product("0N06MRuuX5WQjOvyYDy0O$"), task("EL08")).
belongsTo(product("0N06MRuuX5WQjOvyYDy0O_"), task("EL08")).
belongsTo(product("0N06MRuuX5WQjOvyYDy0OY"), task("CL08D")).
belongsTo(product("0N06MRuuX5WQjOvyYDy0Oa"), task("CL08C")).
belongsTo(product("0N06MRuuX5WQjOvyYDy0Oc"), task("CL08B")).
belongsTo(product("0N06MRuuX5WQjOvyYDy0Oe"), task("CL08A")).
belongsTo(product("0N06MRuuX5WQjOvyYDy0Og"), task("CL08A")).
belongsTo(product("0N06MRuuX5WQjOvyYDy0Oi"), task("CL08C")).
belongsTo(product("0N06MRuuX5WQjOvyYDy0Ok"), task("CL08D")).
belongsTo(product("0N06MRuuX5WQjOvyYDy0OG"), task("CL08C")).
belongsTo(product("0N06MRuuX5WQjOvyYDy0OI"), task("CL08A")).
belongsTo(product("0N06MRuuX5WQjOvyYDy0OK"), task("CL08B")).
belongsTo(product("0N06MRuuX5WQjOvyYDy0OM"), task("CL08B")).
belongsTo(product("0N06MRuuX5WQjOvyYDy0OO"), task("CL08D")).
belongsTo(product("0N06MRuuX5WQjOvyYDy0OQ"), task("CL08A")).
belongsTo(product("0N06MRuuX5WQjOvyYDy0OS"), task("CL08B")).
belongsTo(product("0N06MRuuX5WQjOvyYDy0OU"), task("CL08D")).
belongsTo(product("0N06MRuuX5WQjOvyYDy0O0"), task("CL08C")).
belongsTo(product("0N06MRuuX5WQjOvyYDy0O4"), task("SL09D")).
belongsTo(product("0N06MRuuX5WQjOvyYDy0OF"), task("SL09C")).
belongsTo(product("0N06MRuuX5WQjOvyYDy0Ps"), task("SL09A")).
belongsTo(product("0N06MRuuX5WQjOvyYDy0PX"), task("SL09B")).
belongsTo(product("0N06MRuuX5WQjOvyYDy0MJ"), task("EL09")).
belongsTo(product("0N06MRuuX5WQjOvyYDy0MI"), task("EL09")).
belongsTo(product("0N06MRuuX5WQjOvyYDy0ML"), task("EL09")).
belongsTo(product("0N06MRuuX5WQjOvyYDy0MK"), task("EL09")).
belongsTo(product("0N06MRuuX5WQjOvyYDy0MO"), task("CL09D")).
belongsTo(product("0N06MRuuX5WQjOvyYDy0MQ"), task("CL09C")).
belongsTo(product("0N06MRuuX5WQjOvyYDy0MS"), task("CL09B")).
belongsTo(product("0N06MRuuX5WQjOvyYDy0MU"), task("CL09A")).
belongsTo(product("0N06MRuuX5WQjOvyYDy0M0"), task("CL09A")).
belongsTo(product("0N06MRuuX5WQjOvyYDy0M2"), task("CL09C")).
belongsTo(product("0N06MRuuX5WQjOvyYDy0M4"), task("CL09D")).
belongsTo(product("0N06MRuuX5WQjOvyYDy0M6"), task("CL09C")).
belongsTo(product("0N06MRuuX5WQjOvyYDy0M8"), task("CL09A")).
belongsTo(product("0N06MRuuX5WQjOvyYDy0MA"), task("CL09B")).
belongsTo(product("0N06MRuuX5WQjOvyYDy0MC"), task("CL09B")).
belongsTo(product("0N06MRuuX5WQjOvyYDy0ME"), task("CL09D")).
belongsTo(product("0N06MRuuX5WQjOvyYDy0Nm"), task("CL09A")).
belongsTo(product("0N06MRuuX5WQjOvyYDy0No"), task("CL09B")).
belongsTo(product("0N06MRuuX5WQjOvyYDy0Nq"), task("CL09D")).
belongsTo(product("0N06MRuuX5WQjOvyYDy0Ns"), task("CL09C")).
belongsTo(product("0N06MRuuX5WQjOvyYDy0Nw"), task("SL10D")).
belongsTo(product("0N06MRuuX5WQjOvyYDy0Nb"), task("SL10C")).
belongsTo(product("0N06MRuuX5WQjOvyYDy0Ni"), task("SL10A")).
belongsTo(product("0N06MRuuX5WQjOvyYDy0NN"), task("SL10B")).
belongsTo(product("0N06MRuuX5WQjOvyYDy3lQ"), task("EL10")).
belongsTo(product("0N06MRuuX5WQjOvyYDy3lT"), task("EL10")).
belongsTo(product("0N06MRuuX5WQjOvyYDy3lS"), task("EL10")).
belongsTo(product("0N06MRuuX5WQjOvyYDy3lV"), task("EL10")).
belongsTo(product("0N06MRuuX5WQjOvyYDy3l3"), task("CL10D")).
belongsTo(product("0N06MRuuX5WQjOvyYDy3l5"), task("CL10C")).
belongsTo(product("0N06MRuuX5WQjOvyYDy3l7"), task("CL10B")).
belongsTo(product("0N06MRuuX5WQjOvyYDy3l9"), task("CL10A")).
belongsTo(product("0N06MRuuX5WQjOvyYDy3lB"), task("CL10A")).
belongsTo(product("0N06MRuuX5WQjOvyYDy3lD"), task("CL10C")).
belongsTo(product("0N06MRuuX5WQjOvyYDy3lF"), task("CL10D")).
belongsTo(product("0N06MRuuX5WQjOvyYDy3in"), task("CL10C")).
belongsTo(product("0N06MRuuX5WQjOvyYDy3ip"), task("CL10A")).
belongsTo(product("0N06MRuuX5WQjOvyYDy3ir"), task("CL10B")).
belongsTo(product("0N06MRuuX5WQjOvyYDy3it"), task("CL10B")).
belongsTo(product("0N06MRuuX5WQjOvyYDy3iv"), task("CL10D")).
belongsTo(product("0N06MRuuX5WQjOvyYDy3ix"), task("CL10A")).
belongsTo(product("0N06MRuuX5WQjOvyYDy3iz"), task("CL10B")).
belongsTo(product("0N06MRuuX5WQjOvyYDy3i$"), task("CL10D")).
belongsTo(product("0N06MRuuX5WQjOvyYDy3iX"), task("CL10C")).
belongsTo(product("0N06MRuuX5WQjOvyYDy3ib"), task("SL11D")).
belongsTo(product("0N06MRuuX5WQjOvyYDy3ii"), task("SL11C")).
belongsTo(product("0N06MRuuX5WQjOvyYDy3iN"), task("SL11A")).
belongsTo(product("0N06MRuuX5WQjOvyYDy3iU"), task("SL11B")).
belongsTo(product("0N06MRuuX5WQjOvyYDy3iE"), task("SL12")).
belongsTo(product("0A8QdCd812$xV$VdhPItzP"), task("EX")).
belongsTo(product("0A8QdCd812$xV$VdhPItzI"), task("EX")).



construct(task("CS01"), 0).
resource(task("CS01"), 5).
construct(task("CS02"), 5).
resource(task("CS02"), 5).
construct(task("CS03"), 10).
resource(task("CS03"), 5).
construct(task("CS04"), 15).
resource(task("CS04"), 10).
construct(task("CS05"), 22).
resource(task("CS05"), 3).
construct(task("SL00"), 25).
resource(task("SL00"), 2).
construct(task("EL00"), 27).
resource(task("EL00"), 4).
construct(task("EL01"), 31).
resource(task("EL01"), 4).
construct(task("EL02"), 35).
resource(task("EL02"), 2).
construct(task("EL03"), 37).
resource(task("EL03"), 1).
construct(task("EL04"), 38).
resource(task("EL04"), 1).
construct(task("EL05"), 39).
resource(task("EL05"), 1).
construct(task("EL06"), 40).
resource(task("EL06"), 1).
construct(task("EL07"), 41).
resource(task("EL07"), 1).
construct(task("EL08"), 42).
resource(task("EL08"), 1).
construct(task("EL09"), 43).
resource(task("EL09"), 2).
construct(task("EL10"), 45).
resource(task("EL10"), 2).
construct(task("SL12"), 47).
resource(task("SL12"), 1).
construct(task("SL01A"), 48).
resource(task("SL01A"), 1).
construct(task("SL01B"), 49).
resource(task("SL01B"), 2).
construct(task("SL01C"), 51).
resource(task("SL01C"), 2).
construct(task("SL01D"), 53).
resource(task("SL01D"), 2).
construct(task("CL01A"), 55).
resource(task("CL01A"), 1).
construct(task("CL01B"), 56).
resource(task("CL01B"), 1).
construct(task("CL01C"), 57).
resource(task("CL01C"), 1).
construct(task("CL01D"), 58).
resource(task("CL01D"), 1).
construct(task("SL02A"), 59).
resource(task("SL02A"), 2).
construct(task("SL02B"), 61).
resource(task("SL02B"), 2).
construct(task("SL02C"), 63).
resource(task("SL02C"), 2).
construct(task("SL02D"), 65).
resource(task("SL02D"), 2).
construct(task("CL02A"), 67).
resource(task("CL02A"), 1).
construct(task("CL02B"), 68).
resource(task("CL02B"), 1).
construct(task("CL02C"), 69).
resource(task("CL02C"), 1).
construct(task("CL02D"), 70).
resource(task("CL02D"), 1).
construct(task("SL03A"), 71).
resource(task("SL03A"), 1).
construct(task("SL03B"), 72).
resource(task("SL03B"), 1).
construct(task("SL03C"), 73).
resource(task("SL03C"), 1).
construct(task("SL03D"), 74).
resource(task("SL03D"), 1).
construct(task("CL03A"), 75).
resource(task("CL03A"), 1).
construct(task("CL03B"), 76).
resource(task("CL03B"), 1).
construct(task("CL03C"), 77).
resource(task("CL03C"), 1).
construct(task("CL03D"), 78).
resource(task("CL03D"), 1).
construct(task("SL04A"), 79).
resource(task("SL04A"), 1).
construct(task("SL04B"), 80).
resource(task("SL04B"), 1).
construct(task("SL04C"), 81).
resource(task("SL04C"), 1).
construct(task("SL04D"), 82).
resource(task("SL04D"), 1).
construct(task("CL04A"), 83).
resource(task("CL04A"), 1).
construct(task("CL04B"), 84).
resource(task("CL04B"), 1).
construct(task("CL04C"), 85).
resource(task("CL04C"), 1).
construct(task("CL04D"), 86).
resource(task("CL04D"), 1).
construct(task("SL05A"), 87).
resource(task("SL05A"), 1).
construct(task("SL05B"), 88).
resource(task("SL05B"), 1).
construct(task("SL05C"), 89).
resource(task("SL05C"), 1).
construct(task("SL05D"), 90).
resource(task("SL05D"), 1).
construct(task("CL05A"), 91).
resource(task("CL05A"), 1).
construct(task("CL05B"), 92).
resource(task("CL05B"), 1).
construct(task("CL05C"), 93).
resource(task("CL05C"), 1).
construct(task("CL05D"), 94).
resource(task("CL05D"), 1).
construct(task("SL06A"), 95).
resource(task("SL06A"), 1).
construct(task("SL06B"), 96).
resource(task("SL06B"), 1).
construct(task("SL06C"), 97).
resource(task("SL06C"), 1).
construct(task("SL06D"), 98).
resource(task("SL06D"), 1).
construct(task("CL06A"), 99).
resource(task("CL06A"), 1).
construct(task("CL06B"), 100).
resource(task("CL06B"), 1).
construct(task("CL06C"), 101).
resource(task("CL06C"), 1).
construct(task("CL06D"), 102).
resource(task("CL06D"), 1).
construct(task("SL07A"), 103).
resource(task("SL07A"), 1).
construct(task("SL07B"), 104).
resource(task("SL07B"), 1).
construct(task("SL07C"), 105).
resource(task("SL07C"), 1).
construct(task("SL07D"), 106).
resource(task("SL07D"), 1).
construct(task("CL07A"), 107).
resource(task("CL07A"), 1).
construct(task("CL07B"), 108).
resource(task("CL07B"), 1).
construct(task("CL07C"), 109).
resource(task("CL07C"), 1).
construct(task("CL07D"), 110).
resource(task("CL07D"), 1).
construct(task("SL08A"), 111).
resource(task("SL08A"), 1).
construct(task("SL08B"), 112).
resource(task("SL08B"), 1).
construct(task("SL08C"), 113).
resource(task("SL08C"), 1).
construct(task("SL08D"), 114).
resource(task("SL08D"), 1).
construct(task("CL08A"), 115).
resource(task("CL08A"), 1).
construct(task("CL08B"), 116).
resource(task("CL08B"), 1).
construct(task("CL08C"), 117).
resource(task("CL08C"), 1).
construct(task("CL08D"), 118).
resource(task("CL08D"), 1).
construct(task("SL09A"), 119).
resource(task("SL09A"), 2).
construct(task("SL09B"), 121).
resource(task("SL09B"), 2).
construct(task("SL09C"), 123).
resource(task("SL09C"), 2).
construct(task("SL09D"), 113).
resource(task("SL09D"), 2).
construct(task("CL09A"), 115).
resource(task("CL09A"), 1).
construct(task("CL09B"), 116).
resource(task("CL09B"), 1).
construct(task("CL09C"), 117).
resource(task("CL09C"), 1).
construct(task("CL09D"), 118).
resource(task("CL09D"), 1).
construct(task("SL10A"), 119).
resource(task("SL10A"), 2).
construct(task("SL10B"), 121).
resource(task("SL10B"), 2).
construct(task("SL10C"), 123).
resource(task("SL10C"), 2).
construct(task("SL10D"), 125).
resource(task("SL10D"), 2).
construct(task("CL10A"), 127).
resource(task("CL10A"), 1).
construct(task("CL10B"), 128).
resource(task("CL10B"), 1).
construct(task("CL10C"), 129).
resource(task("CL10C"), 1).
construct(task("CL10D"), 130).
resource(task("CL10D"), 1).
construct(task("SL11A"), 131).
resource(task("SL11A"), 2).
construct(task("SL11B"), 133).
resource(task("SL11B"), 2).
construct(task("SL11C"), 135).
resource(task("SL11C"), 2).
construct(task("SL11D"), 137).
resource(task("SL11D"), 2).



is_a(product("0N06MRuuX5WQjOvzkDy3l0"), opening).
containedIn(product("0N06MRuuX5WQjOvzkDy3l0"), storey(10)).
is_a(product("0N06MRuuX5WQjOvzkDy3lU"), opening).
containedIn(product("0N06MRuuX5WQjOvzkDy3lU"), storey(10)).
is_a(product("0N06MRuuX5WQjOvzkDy3l1"), opening).
containedIn(product("0N06MRuuX5WQjOvzkDy3l1"), storey(10)).
is_a(product("0N06MRuuX5WQjOvzkDy0MP"), opening).
containedIn(product("0N06MRuuX5WQjOvzkDy0MP"), storey(9)).
is_a(product("0N06MRuuX5WQjOvzkDy0MN"), opening).
containedIn(product("0N06MRuuX5WQjOvzkDy0MN"), storey(9)).
is_a(product("0N06MRuuX5WQjOvzkDy0MM"), opening).
containedIn(product("0N06MRuuX5WQjOvzkDy0MM"), storey(9)).
is_a(product("0N06MRuuX5WQjOvzkDy0OZ"), opening).
containedIn(product("0N06MRuuX5WQjOvzkDy0OZ"), storey(8)).
is_a(product("0N06MRuuX5WQjOvzkDy0OX"), opening).
containedIn(product("0N06MRuuX5WQjOvzkDy0OX"), storey(8)).
is_a(product("0N06MRuuX5WQjOvzkDy0OW"), opening).
containedIn(product("0N06MRuuX5WQjOvzkDy0OW"), storey(8)).
is_a(product("0N06MRuuX5WQjOvzkDy0TC"), opening).
containedIn(product("0N06MRuuX5WQjOvzkDy0TC"), storey(7)).
is_a(product("3eORFXfAH04e_QSxfI$ODY"), opening).
containedIn(product("3eORFXfAH04e_QSxfI$ODY"), storey(1)).
is_a(product("3eORFXfAH04e_QSxfI$ODX"), opening).
containedIn(product("3eORFXfAH04e_QSxfI$ODX"), storey(1)).
is_a(product("3eORFXfAH04e_QSxfI$ODZ"), opening).
containedIn(product("3eORFXfAH04e_QSxfI$ODZ"), storey(1)).
is_a(product("3eORFXfAH04e_QSxfI$O1C"), opening).
containedIn(product("3eORFXfAH04e_QSxfI$O1C"), storey(2)).
is_a(product("3eORFXfAH04e_QSxfI$O06"), opening).
containedIn(product("3eORFXfAH04e_QSxfI$O06"), storey(2)).
is_a(product("3eORFXfAH04e_QSxfI$O9p"), opening).
containedIn(product("3eORFXfAH04e_QSxfI$O9p"), storey(2)).
is_a(product("3eORFXfAH04e_QSxfI$ODp"), opening).
containedIn(product("3eORFXfAH04e_QSxfI$ODp"), storey(3)).
is_a(product("3eORFXfAH04e_QSxfI$ODq"), opening).
containedIn(product("3eORFXfAH04e_QSxfI$ODq"), storey(3)).
is_a(product("3eORFXfAH04e_QSxfI$ODo"), opening).
containedIn(product("3eORFXfAH04e_QSxfI$ODo"), storey(3)).
is_a(product("3eORFXfAH04e_QSxfI$OD1"), opening).
containedIn(product("3eORFXfAH04e_QSxfI$OD1"), storey(4)).
is_a(product("3eORFXfAH04e_QSxfI$OD2"), opening).
containedIn(product("3eORFXfAH04e_QSxfI$OD2"), storey(4)).
is_a(product("3eORFXfAH04e_QSxfI$OD0"), opening).
containedIn(product("3eORFXfAH04e_QSxfI$OD0"), storey(4)).
is_a(product("3eORFXfAH04e_QSxfI$ODG"), opening).
containedIn(product("3eORFXfAH04e_QSxfI$ODG"), storey(5)).
is_a(product("3eORFXfAH04e_QSxfI$ODH"), opening).
containedIn(product("3eORFXfAH04e_QSxfI$ODH"), storey(5)).
is_a(product("3eORFXfAH04e_QSxfI$ODV"), opening).
containedIn(product("3eORFXfAH04e_QSxfI$ODV"), storey(5)).
is_a(product("0N06MRuuX5WQjOvzkDy0TA"), opening).
containedIn(product("0N06MRuuX5WQjOvzkDy0TA"), storey(7)).
is_a(product("3eORFXfAH04e_QSxfI$OIW"), opening).
containedIn(product("3eORFXfAH04e_QSxfI$OIW"), storey(6)).
is_a(product("3eORFXfAH04e_QSxfI$OIX"), opening).
containedIn(product("3eORFXfAH04e_QSxfI$OIX"), storey(6)).
is_a(product("3eORFXfAH04e_QSxfI$OIl"), opening).
containedIn(product("3eORFXfAH04e_QSxfI$OIl"), storey(6)).
is_a(product("0N06MRuuX5WQjOvzkDy0TD"), opening).
containedIn(product("0N06MRuuX5WQjOvzkDy0TD"), storey(7)).

"""

In [251]:
class QSTR:

    def __init__(self, geoms, rel, assigns):
        self._geoms = geoms
        self._rel = rel
        self._assigns = assigns

    def create_geom(self, r):
        return _create_geom(r)

    #return Polygon.Polygon(repr[0])

    def add_geom(self, obj_id, r):
        geom_id = len(self._geoms) + 1
        self._assigns.setdefault(obj_id, geom_id)
        self._geoms.setdefault(geom_id, self.create_geom(r))
        return geom_id

    def get_geom_(self, id):
        if id in db.keys():
            r = db[id]
            return self.add_geom(id, r)
        else:
            return self._geoms[id]

    def get_geom(self, id):

        assigns = self._assigns
        #print(assigns)
        if id in assigns.keys():
            geom_id = assigns.get(id)
            return geom_id
        else:
            if id in db.keys():
                #print("repr")
                r = db[id]
                return self.add_geom(id, r)
            else:
                raise RuntimeError("geometric data not available")

    def union_(self, shapes):
        if len(shapes) == 0:
            return "void"
        res = shapes[0]
        if len(shapes) > 1:
            res += self.union_(shapes[1:])
        return res

    def union(self, l):
        if len(l) == 0:
            return "void"
        shapes_ = [self._geoms.get(elt) for elt in l]
        shapes = [shape for shape in shapes_ if shape != "void"]
        return self.union_(shapes)

    def diff(self, geom_id1, geom_id2):
        shape1 = self._geoms.get(geom_id1)
        shape2 = self._geoms.get(geom_id2)
        if shape1 == "void":
            new_shape = "void"
        elif shape2 == "void":
            new_shape = shape1
        else:
            new_shape = shape1 - shape2
        return new_shape

    def buffer(self, geom_id, value):
        shape = self._geoms.get(geom_id)
        if shape == "void":
            new_shape = "void"
        else:
            new_shape = _buffer(shape, value)
        return new_shape

    def intersect_(self, shapes):
        if len(shapes) == 0:
            return "void"
        res = shapes[0]
        if len(shapes) > 1:
            res &= self.intersect_(shapes[1:])
        return res

    def intersect(self, l):
        if len(l) == 0:
            return "void"
        shapes = [self._geoms.get(elt) for elt in l]
        if "void" in shapes:
            return "void"
        else:
            return self.intersect_(shapes)

    def _draw(self, l, s):
        shapes = [self._geoms.get(elt) for elt in l]
        s1, s2, s3 = shapes[0], shapes[1], shapes[2]
        n1, n2, n3 = './pics/' + s + '_f.svg', './pics/' + s + '_h.svg', './pics/' + s + '_m.svg'
        n = './pics/' + s + '.svg'
        if s1 != "void" and s2 != "void" and s3 != "void":
            Polygon.IO.writeSVG(n1, (s1,))
            Polygon.IO.writeSVG(n2, (s2,))
            Polygon.IO.writeSVG(n3, (s3,))
            Polygon.IO.writeSVG(n, (s3, s1, s2))

    def _in(self, h, heads):
        for head in heads:
            if head in h:
                return True
        return False

    def _category(self, id):
        for name in spatial_function_names + spatial_artefact_names:
            if id.startswith(name):
                return id
        try:
            return float(id)
        except:
            return '"' + id + '"'

    def _conv(self, id):
        if id.startswith('"'):
            return id[1:-1]
        else:
            return id

    def _convert_draw(self, id):
        ids = ast.literal_eval(id)
        ids = [self._category(id) for id in ids]
        ids = [self._conv(id) for id in ids]
        geom_ids = [self.get_geom(id) for id in ids]
        return geom_ids

    def _convert(self, id):
        ids = ast.literal_eval(id)
        ids = [self._category(id) for id in ids]
        geom_ids = [self.get_geom(id) for id in ids]
        return geom_ids

    def _convert_buffer(self, id):

        ids = ast.literal_eval(id)
        ids = [self._category(id) for id in ids]
        geom_id = self.get_geom(ids[0])
        return geom_id, ids[1]

    def _convert_diff(self, id):

        ids = ast.literal_eval(id)
        ids = [self._category(id) for id in ids]
        geom_ids = [self.get_geom(id) for id in ids]
        return geom_ids

    def _evaluate(self, head_atoms):

        if len(head_atoms) == 0:
            return
        heads = [t[3] for t in head_atoms if len(t) > 3]
        head_atoms_ = []
        for h in head_atoms:
            if not self._in(h[2], heads):

                #print(h)
                if h[0] == "diff":
                    res = h[3]
                    if res not in self._assigns.keys():
                        geom_ids = self._convert_diff(h[2])
                        new_id = 'diff(' + ','.join([str(id) for id in geom_ids]) + ')'
                        new_shape = self.diff(geom_ids[0], geom_ids[1])
                        self._geoms.setdefault(new_id, new_shape)
                        self._assigns.setdefault(res, new_id)
                        self._geoms.setdefault(res, new_shape)

                if h[0] == "union":
                    res = h[3]
                    if res not in self._assigns.keys():
                        geom_ids = self._convert(h[2])
                        new_id = 'union(' + ','.join([str(id) for id in geom_ids]) + ')'
                        new_shape = self.union(geom_ids)
                        self._geoms.setdefault(new_id, new_shape)
                        self._assigns.setdefault(res, new_id)

                if h[0] == "intersect":
                    res = h[3]
                    if res not in self._assigns.keys():
                        geom_ids = self._convert(h[2])
                        new_id = 'intersect(' + ','.join([str(id) for id in geom_ids]) + ')'
                        new_shape = self.intersect(geom_ids)
                        self._geoms.setdefault(new_id, new_shape)
                        self._assigns.setdefault(res, new_id)

                if h[0] == "buffer":
                    res = h[3]
                    if res not in self._assigns.keys():
                        geom_id, val = self._convert_buffer(h[2])
                        new_id = 'buffer(' + ','.join([geom_id, str(val)]) + ')'
                        new_shape = self.buffer(geom_id, val)
                        self._geoms.setdefault(new_id, new_shape)
                        self._assigns.setdefault(res, new_id)

                if h[0] == "draw":
                    geom_ids = self._convert_draw(h[2])
                    shapes = [self._geoms.get(geom_id) for geom_id in geom_ids]
                    shapes = [shape for shape in shapes if shape != "void"]
                    if len(shapes) > 0:
                        file_name = './pics/' + h[2][2:-2] + '.svg'
                        Polygon.IO.writeSVG(file_name, shapes)
            else:
                #print(2, h)
                head_atoms_.append(h)
        return self._evaluate(head_atoms_)

    def topology(self, x, y):
        print_msg("Heuristics : TOPOLOGICAL", 1)
        if x < y:
            return "part_of"
        else:
            return "overlaps"

    def _compute(self, body_atoms):
        if len(body_atoms) == 0:
            return
        for b in body_atoms:
            if b[0] in spatial_relation_names:
                r1, r2 = b[2], b[3]
                x, y = self._geoms.get(r1), self._geoms.get(r2)
                if (r1, r2) not in self._rel.keys():
                    self._rel.setdefault((r1, r2), self.topology(x, y))
        return

    def get_assignment(self):
        return self._geoms

    def get_rel(self):
        return self._rel

    def get_repr(self):
        return self._assigns


# Transformer

In [252]:
class GACTransformer(clingo.ast.Transformer):

    def __init__(self, builder, check):
        self._builder = builder
        self._state = "guess"
        self._check = check

    def add(self, stm):
        if stm.ast_type is clingo.ast.ASTType.Program:
            if stm.name == "check" and not stm.parameters:
                stm.name = "check"
                self._state = "check"
            elif stm.name in ("base", "guess") and not stm.parameters:
                self._state = "guess"
            else:
                raise RuntimeError("unexpected program part")

        else:
            if self._state == "guess":
                self._builder.add(self.visit(stm))
            else:
                self._check.append(self.visit(stm))

    def visit_Rule(self, stm):
        head = stm.head
        body = stm.body
        loc = stm.location
        if head.ast_type is clingo.ast.ASTType.TheoryAtom:
            new_head = self.visit_TheoryAtom(head, "head")
        else:
            new_head = head

        new_body = []
        for lit in body:
            if lit.atom.ast_type is clingo.ast.ASTType.TheoryAtom:
                lit.atom = self.visit_TheoryAtom(lit.atom, "body")
            new_body.append(lit)
        return clingo.ast.Rule(loc, new_head, new_body)

    def visit_TheoryAtom(self, atom, loc):
        term = atom.term
        if term.name in theory_atom_names and not term.arguments:
            atom.term = clingo.ast.Function(term.location, term.name,
                                            [clingo.ast.Function(term.location, loc, [], False)], False)
        return atom


# Clingo Specifics

In [253]:
THEORY = """
#theory spatial {
	constant  { - : 1, unary };
	variable  { - : 1, unary };
	spatial_term {
		- : 3 , unary ;
		** : 2 , binary, right ;
		* : 1 , binary, left ;
		/ : 1 , binary , left ;
		\\ : 1 , binary , left ;
		+ : 0 , binary, left ;
		- : 0 , binary, left };
	&topology/1 : spatial_term, any;
	&union/1 : spatial_term, {=}, constant, any;
	&diff/1 : spatial_term, {=}, constant, any ;
	&buffer/1 : spatial_term, {=}, constant, any ;
	&intersect/1 : spatial_term, {=}, constant, any ;
	&draw/1 : spatial_term, any
}.
"""

In [254]:
program = """

timestep(X, Te) :-
	is_a(product(X), _),
	belongsTo(product(X), task(L)),
	construct(task(L), Ts),
	resource(task(L), Td),
	Te = Ts + Td.

timestep(Y, Te) :-
	is_a(product(Y), _),
	voids(product(Y), product(X)),
	belongsTo(product(X), task(L)),
	construct(task(L), Ts),
	resource(task(L), Td),
	Te = Ts + Td.


product(T, X) :-
	is_a(product(X), T), type(T), timestep(X, Te), time_now(T0), Te <= T0.

on(S, X) :-
	containedIn(product(X), storey(S)), timestep(X, Te), time_now(T0), Te <= T0.

type(slab; wall; opening; column).
storey(S) :- product(slab, X), on(S, X).

#show storey/1.
#show time_now/1.

&union{Id : product(T, Id), on(S, Id)} = union(T, S) :-
	type(T), storey(S).

&buffer{union(slab, S), "0.01"} = buffer(union(slab, S)) :-
	storey(S), &union{Id : product(slab, Id), on(S, Id)} = union(slab, S).

&buffer{union(wall, S), "0.01"} = buffer(union(wall, S)) :-
	storey(S), &union{Id : product(wall, Id), on(S, Id)} = union(wall, S).

&buffer{union(column, S), "0.02"} = buffer(union(column, S)) :-
	storey(S), &union{Id : product(column, Id), on(S, Id)} = union(column, S).

&diff{buffer(union(slab, S)) - buffer(union(column, S))} = walkable_surface(S) :-
	storey(S), &buffer{union(wall, S), "0.01"} = buffer(union(wall, S)),
	&buffer{union(column, S), "0.02"} = buffer(union(column, S)).

&intersect{buffer(union(wall, S)); union(opening, S)} = transition_space(S) :-
	storey(S), &buffer{union(wall, S), "0.01"} = buffer(union(wall, S)),
	&union{Id : product(opening, Id), on(S, Id)} = union(opening, S).

&union{buffer(union(slab, S)); buffer(union(wall, S))} = constructed_surface(S) :-
	storey(S),
	&buffer{union(slab, S), "0.01"} = buffer(union(slab, S)),
	&buffer{union(wall, S), "0.01"} = buffer(union(wall, S)).

&union{constructed_surface(S) : storey(S)} = union(constructed_surface) :-
	storey(S), &union{buffer(union(slab, S)); buffer(union(wall, S))} = constructed_surface(S).

&buffer{union(constructed_surface), "10.0"} = site :-
	storey(S), &union{constructed_surface(S) : storey(S)} = union(constructed_surface).

&diff{site - constructed_surface(S)} = fall_space(S) :-
	storey(S), &buffer{union(constructed_surface), "10.0"} = site,
	&union{buffer(union(slab, S)); buffer(union(wall, S))} = constructed_surface(S).

&buffer{fall_space(S), "0.05"} = buffer(fall_space(S)) :-
	storey(S), &diff{site - constructed_surface(S)} = fall_space(S).

&union{walkable_surface(S); transition_space(S)} = movement_space(S) :-
	storey(S),
	&diff{buffer(union(slab, S)) - buffer(union(column, S))} = walkable_surface(S),
	&intersect{buffer(union(wall, S)); union(opening, S)} = transition_space(S).

&intersect{movement_space(S); buffer(fall_space(S))} = hazard_space(S) :-
	storey(S),
	&union{walkable_surface(S); transition_space(S)} = movement_space(S),
	&buffer{fall_space(S), "0.05"} = buffer(fall_space(S)).

"""

In [255]:
gac = """

#program guess.

1{time_now(1..122)}1.


"""

# Executing

In [256]:
ctl = clingo.Control()
ctl.configuration.solve.models = 0

In [257]:
check = []
with ProgramBuilder(ctl) as builder:
    trans = GACTransformer(builder, check)
    clingo.ast.parse_string(gac, trans.add)
    clingo.ast.parse_string(program, trans.add)

In [258]:
gac_propagator = GACPropagator(check)
ctl.register_propagator(gac_propagator)

In [259]:
ctl.add('base', [], THEORY)
#ctl.add('base', [], data)
ctl.ground([('base', [])])

<string>:4:2-21: info: atom does not occur in any rule head:
  is_a(product(#X0),#P1)

<string>:5:2-32: info: atom does not occur in any rule head:
  belongsTo(product(X),task(L))

<string>:6:2-24: info: atom does not occur in any rule head:
  construct(task(L),Ts)

<string>:7:2-23: info: atom does not occur in any rule head:
  resource(task(L),Td)

<string>:12:2-31: info: atom does not occur in any rule head:
  voids(product(Y),product(X))

<string>:13:2-32: info: atom does not occur in any rule head:
  belongsTo(product(X),task(L))

<string>:14:2-24: info: atom does not occur in any rule head:
  construct(task(L),Ts)

<string>:15:2-23: info: atom does not occur in any rule head:
  resource(task(L),Td)

<string>:20:2-21: info: atom does not occur in any rule head:
  is_a(product(X),T)

<string>:23:2-36: info: atom does not occur in any rule head:
  containedIn(product(X),storey(S))



In [260]:
with ctl.solve(yield_=True, on_model=gac_propagator.on_model) as solve_handle:
    models = []
    for model in solve_handle:
        symbols = sorted(model.symbols(shown=True))
        models.append(model)
        print("Answer {}:".format(model.number), end=' ')
        print("{",
              '\n'.join(map(str, symbols)), "}", sep='\n')
    solve_result = solve_handle.get()
    print(solve_result, end='')
    if models:
        print(" {}{}".format(len(models), '' if solve_result.exhausted else '+'))


assignment : 
Answer 1: {
time_now(1)
}
assignment : 
Answer 2: {
time_now(2)
}
assignment : 
Answer 3: {
time_now(3)
}
assignment : 
Answer 4: {
time_now(4)
}
assignment : 
Answer 5: {
time_now(5)
}
assignment : 
Answer 6: {
time_now(6)
}
assignment : 
Answer 7: {
time_now(7)
}
assignment : 
Answer 8: {
time_now(8)
}
assignment : 
Answer 9: {
time_now(9)
}
assignment : 
Answer 10: {
time_now(10)
}
assignment : 
Answer 11: {
time_now(11)
}
assignment : 
Answer 12: {
time_now(12)
}
assignment : 
Answer 13: {
time_now(13)
}
assignment : 
Answer 14: {
time_now(14)
}
assignment : 
Answer 15: {
time_now(15)
}
assignment : 
Answer 16: {
time_now(16)
}
assignment : 
Answer 17: {
time_now(17)
}
assignment : 
Answer 18: {
time_now(18)
}
assignment : 
Answer 19: {
time_now(19)
}
assignment : 
Answer 20: {
time_now(20)
}
assignment : 
Answer 21: {
time_now(21)
}
assignment : 
Answer 22: {
time_now(22)
}
assignment : 
Answer 23: {
time_now(23)
}
assignment : 
Answer 24: {
time_now(24)
}
assignment

In [261]:
gac_propagator